<h1> Machine Learning w chmurze - Cloud ML </h1>

W tym notebooku pokażemy jak przenieść prosty model Tensorflow do GCP i uruchomić na nim predykcje

<h2> Predykcje na podstawie tekstu </h2>

<b>Źródło danych</b>: Yelp Restaurant Reviews (https://www.yelp.com/dataset/challenge)

Dataset zawiera między innymi informacje o restauracjach oraz opinie klientów

Zadaniem jest przewidzenie czy restauracje przejdą inspekcje (amerykańskiego) sanepidu na podstawie opinii gości oraz dodatkowych informacji takich jak lokalizacja i rodzaje kuchni serwowanych w restauracji.

<h2> Ustawienie zmiennych środowiskowych, import bibliotek </h2>

In [176]:
import warnings
warnings.filterwarnings('ignore')

In [177]:
BUCKET = 'dswbiznesie'
PROJECT = 'dswbiznesie'
REGION = 'europe-west1'
REPO = '/content/datalab/dswbiznesie'

In [178]:
import os
os.environ['BUCKET'] = BUCKET
os.environ['PROJECT'] = PROJECT
os.environ['REGION'] = REGION
os.environ['REPO'] = REPO

In [179]:
%datalab project set -p $PROJECT

In [180]:
%%javascript
$.getScript('https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js')

<IPython.core.display.Javascript object>

In [181]:
import google.datalab.ml as ml
import tensorflow as tf
import apache_beam as beam
import shutil
import datetime
from apache_beam.io.gcp.internal.clients import bigquery
import pandas as pd
import google.datalab.bigquery as bq
import numpy as np
import random
import string
import unicodedata
import sys
#print tf.__version__
import warnings
warnings.filterwarnings('ignore')

<h2> Dane źródłowe </h2>


Dataset pobrany ze strony Yelp zawiera następujące pliki

In [8]:
!gsutil -q ls -l gs://$BUCKET/rawdata/hygiene

 103941968  2017-11-06T00:52:39Z  gs://dswbiznesie/rawdata/hygiene/hygiene.dat
    831242  2017-11-03T01:14:18Z  gs://dswbiznesie/rawdata/hygiene/hygiene.dat.additional
    159046  2017-11-03T01:14:17Z  gs://dswbiznesie/rawdata/hygiene/hygiene.dat.labels
TOTAL: 3 objects, 104932256 bytes (100.07 MiB)


* <b>hygiene.dat</b>: każda linia zawiera połączone opinie klientów danej restauracji
* <b>hygiene.dat.labels</b>: dla pierwszych 546 linii przypisana jest dodatkowe pole w którym 0 oznacza to że restauracja przeszła inspekcje, 1 to że restauracja <b>nie</b> przeszła inspekcji. Reszta wpisów posiada wpis "[None]" i nie będą brane pod uwagę przy obliczaniu modelu
* <b>hygiene.dat.additional</b>: plik CSV gdzie w pierwszym polu znajduje się lista oferowanych rodzajów kuchnii, w drugim kod pocztowy restauracji który można uznać za przybliżenie lokalizacji restauracji. W trzecim polu znajduje się liczba opinii, w czwartym średnia ocena ( w skali 0-5, gdzie 5 oznacza ocene najlepszą).

<h2> Feature engineering używając Apache Beam i BigQuery</h2>

Dane źródłowe należy przetwrzyć i dostosować do postaci której będzie można łatwo użyć do uczenia i ewaluacji modelu. 
Najwygodniejszym choć nie najtańszym rozwiązaniem jest załadowanie danych do BigQuery.

Odpowiednim narzędziem do tego zadania jest Apache Beam i jego implementacja - Google Dataflow.
Job Dataflow uruchamiany jest w chmurze. Jego przebieg można śledzić w Konsoli GCP (https://console.cloud.google.com/dataflow).
Uruchomienie joba trwa powyżej minuty.

In [14]:
def create_record(rest_tuple):
    #print(rest_tuple)
    identity = rest_tuple[0]
    reviews = rest_tuple[1]['reviews_kv'][0]
    inspection_result = int(rest_tuple[1]['labels_kv'][0]) if rest_tuple[1]['labels_kv'][0] != "[None]" else None
    categories_temp = rest_tuple[1]['attributes_kv'][0].split("\"")
    categories = ",".join([ x.replace('\'', '').replace('[','').replace(']','').strip() for x in categories_temp[1].split(',')])
    #categories = categories_temp[1].replace('\'', '').replace('[','').replace(']','')
    attributes_temp = categories_temp[2].split(",")
    zip_code = attributes_temp[1]
    review_count = int(attributes_temp[2])
    avg_rating = float(attributes_temp[3])
    
    return { 
        'identity': identity, 
        'reviews': reviews,
        'inspection_result': inspection_result,
        'categories': categories,
        'zip_code': zip_code,
        'review_count': review_count,
        'avg_rating': avg_rating
    }

def preprocess(RUNNER,BUCKET,BIGQUERY_TABLE):
    job_name = 'hygiene-ftng' + '-' + datetime.datetime.now().strftime('%y%m%d-%H%M%S')
    print 'Launching Dataflow job {} ... hang on'.format(job_name)
    OUTPUT_DIR = 'gs://{0}/data/hygiene/'.format(BUCKET)
    options = {
        'staging_location': os.path.join(OUTPUT_DIR, 'tmp', 'staging'),
        'temp_location': os.path.join(OUTPUT_DIR, 'tmp'),
        'job_name': 'hygiene-ftng' + '-' + datetime.datetime.now().strftime('%y%m%d-%H%M%S'),
        'project': PROJECT,
        'region': 'europe-west1',
        'teardown_policy': 'TEARDOWN_ALWAYS',
        'no_save_main_session': True
    }
    opts = beam.pipeline.PipelineOptions(flags=[], **options)
    p = beam.Pipeline(RUNNER, options=opts)
    
    # Adding table definition
    table_schema = bigquery.TableSchema()
    
    # Fields definition
    identity_schema = bigquery.TableFieldSchema()
    identity_schema.name = 'identity'
    identity_schema.type = 'integer'
    identity_schema.mode = 'required'
    table_schema.fields.append(identity_schema)
    
    
    reviews_schema = bigquery.TableFieldSchema()
    reviews_schema.name = 'reviews'
    reviews_schema.type = 'string'
    reviews_schema.mode = 'required'
    table_schema.fields.append(reviews_schema)

    inspection_result_schema = bigquery.TableFieldSchema()
    inspection_result_schema.name = 'inspection_result'
    inspection_result_schema.type = 'integer'
    inspection_result_schema.mode = 'nullable'
    table_schema.fields.append(inspection_result_schema)
    
    categories_schema = bigquery.TableFieldSchema()
    categories_schema.name = 'categories'
    categories_schema.type = 'string'
    categories_schema.mode = 'required'
    table_schema.fields.append(categories_schema)
    
    zip_code_schema = bigquery.TableFieldSchema()
    zip_code_schema.name = 'zip_code'
    zip_code_schema.type = 'string'
    zip_code_schema.mode = 'required'
    table_schema.fields.append(zip_code_schema)
    
    review_count_schema = bigquery.TableFieldSchema()
    review_count_schema.name = 'review_count'
    review_count_schema.type = 'integer'
    review_count_schema.mode = 'required'
    table_schema.fields.append(review_count_schema)
    
    avg_rating_schema = bigquery.TableFieldSchema()
    avg_rating_schema.name = 'avg_rating'
    avg_rating_schema.type = 'float'
    avg_rating_schema.mode = 'required'
    table_schema.fields.append(avg_rating_schema)
    
    #processing logic
    reviews = p | 'Read Reviews' >> beam.io.ReadFromText('gs://{0}/rawdata/hygiene/hygiene.dat'.format(BUCKET))
    labels = p | 'Read Labels' >> beam.io.ReadFromText('gs://{0}/rawdata/hygiene/hygiene.dat.labels'.format(BUCKET))
    attributes = p | 'Read Attributes' >> beam.io.ReadFromText('gs://{0}/rawdata/hygiene/hygiene.dat.additional'.format(BUCKET))
    
    reviews_kv = reviews | 'Map Reviews to KV' >> beam.Map(lambda x: (x.split(",")[0], ",".join(x.split(",")[1:]).replace("|","")))
    labels_kv = labels | 'Map Labels to KV' >> beam.Map(lambda x: (x.split(",")[0], x.split(",")[1]))
    attributes_kv = attributes | 'Map Attributes to KV' >> beam.Map(lambda x: (x.split(",")[0], x))
    
    restaurants = (
        {'reviews_kv': reviews_kv, 'labels_kv': labels_kv, 'attributes_kv': attributes_kv}
        | 'CoGroup By Restaurant Key' >> beam.CoGroupByKey())
    
    records = restaurants | 'Create Records' >> beam.Map(create_record)
    
    records | 'Write to BigQuery' >> beam.io.Write(
        beam.io.BigQuerySink(
            BIGQUERY_TABLE,
            schema=table_schema,
            create_disposition=beam.io.BigQueryDisposition.CREATE_IF_NEEDED,
            write_disposition=beam.io.BigQueryDisposition.WRITE_TRUNCATE))
    return p.run()

<b> Uruchomienie etl </b>

In [19]:
bigquery_dataset = "{}:{}.hygiene".format(PROJECT,PROJECT)
#preprocess('DirectRunner',BUCKET, bigquery_dataset)
job = preprocess('Dataflow', BUCKET, bigquery_dataset)

Launching Dataflow job hygiene-ftng-171118-193842 ... hang on


<h2> Przygotowanie danych do trenowania modelu </h2>

<b> Pobranie danych do DataFrame (pandas) </b>

In [146]:
query="""
SELECT
  #identity,
  reviews,
  inspection_result ,
  categories,
  zip_code,
  review_count,
  avg_rating
FROM
  `dswbiznesie.hygiene`
WHERE
  inspection_result IS NOT NULL
"""

<b> Podzial danych na zestaw treningowy i ewaluacyjny </b>

In [150]:
traindf = bq.Query(query + " AND MOD(ABS(FARM_FINGERPRINT(reviews)),4) > 0").execute().result().to_dataframe()
evaldf  = bq.Query(query + " AND MOD(ABS(FARM_FINGERPRINT(reviews)),4) = 0").execute().result().to_dataframe()
traindf.head()

,reviews,inspection_result,categories,zip_code,review_count,avg_rating
0,I went here on a first date which was a tad u...,0,Restaurants,98101,9,3.111111
1,Not impressive. While the decor is pretty gar...,0,Restaurants,98101,3,2.666667
2,Please note that this review only stems from ...,0,"Irish,Restaurants",98101,37,3.302326
3,I grew up in the central valley of California...,0,"Mexican,Restaurants",98101,3,3.333333
4,I'm convinced this place needs to be on Guy F...,0,"Mexican,Restaurants",98101,15,3.200000


In [96]:
traindf['inspection_result'].value_counts()

0    205
1    203
Name: inspection_result, dtype: int64

In [97]:
evaldf['inspection_result'].value_counts()

1    70
0    68
Name: inspection_result, dtype: int64

In [151]:
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer

def preprocess_text(text):
  cleanedtxt = text.replace("&#160;"," ").lower()
  tokenizer = RegexpTokenizer(r'\w+')
  tokens = tokenizer.tokenize(cleanedtxt)
  stop = stopwords.words('english')
  nostoptokens = [i for i in tokens if i not in stop]
  snowball = SnowballStemmer('english')
  stems = [snowball.stem(i) for i in nostoptokens]
  return ' '.join(stems)

In [155]:
traindf['reviews'] = traindf['reviews'].apply(lambda x: preprocess_text(x))
evaldf['reviews'] = evaldf['reviews'].apply(lambda x: preprocess_text(x))

In [156]:
traindf.to_csv('train.csv', header=False, index=False, encoding='utf-8', sep='|')
evaldf.to_csv('eval.csv', header=False, index=False, encoding='utf-8', sep='|')

In [157]:
!head -3 train.csv

went first date tad uncomfort anyway went 8 00pm saturday music realli loud amp techno ish probabl ok like total alon haha ok enough mess coupl martini stoli dirti oliv ok like 3 oliv 2 c mon stingi kettl one w lemon said water realli know tri bad place definit date place chair w tall back hard atmospher nice modern servic great drink ok ladi got dress went w hotel bar last friday night place definit swanki prepar come dress impress cool side note fantasia barrino sit back tabl entourag crew tri hard stare much drink pretti spendi luckili happi hr start 10 pm mon fri enjoy tasti app definit tri sloppi joe like pull pork slider warn may intrigu sexi fri noth sexi sexi fri breath devour like limit garlic fri intak game safeco field needless say bit paranoid rest even get close person anyon confus place peopl seem flock yes chic cozi insid hotel bar popular seattl total conveni bf tire ventur downtown decid give w bar tri sinc could stumbl back upstair need signatur cocktail defin pricey 

In [30]:
!wc -l *.csv

    138 eval.csv
    408 train.csv
    408 vocab.csv
    954 total


In [31]:
%bash
gsutil cp *.csv gs://${BUCKET}/data/

Copying file://eval.csv [Content-Type=text/csv]...
Copying file://train.csv [Content-Type=text/csv]...
Copying file://vocab.csv [Content-Type=text/csv]...
- [3 files][  7.6 MiB/  7.6 MiB]                                                
Operation completed over 3 objects/7.6 MiB.                                      


In [32]:
!gsutil -q ls -l gs://$BUCKET/data

         0  2017-11-03T00:08:16Z  gs://dswbiznesie/data/
   1242719  2017-11-18T19:42:33Z  gs://dswbiznesie/data/eval.csv
   3353088  2017-11-18T19:42:34Z  gs://dswbiznesie/data/train.csv
   3353088  2017-11-18T19:42:34Z  gs://dswbiznesie/data/vocab.csv
     39652  2017-11-11T18:14:43Z  gs://dswbiznesie/data/vocab_words
                                 gs://dswbiznesie/data/hygiene/
TOTAL: 5 objects, 7988547 bytes (7.62 MiB)


<h2> Model Tensorflow </h2>

Właściwy model tensorflow znajduje się w pliku <b>model.py</b> a definicja joba Cloud ML w pliku <b>task.py</b>
Kod poniżej ma za zadanie zilutrować działanie kodu tensorflow.

In [33]:
import tensorflow as tf
from tensorflow.contrib import lookup
from tensorflow.python.platform import gfile

print tf.__version__
MAX_DOCUMENT_LENGTH = 100000  
PADWORD = 'ZYXW'

# vocabulary
lines = ['This might be the best "taco" truck on the planet. Hidden between a smoke shop and The Home Depot, this semi permanent, stylish and clean cafe on wheels.', 
         'Im always looking for a good place to sing karaoke. This is one of them. Drinks are cheap. Food is delicious. And its laid back and fun. I shall return!', 
         'Crazy Man just Crazy there are like 3 different places called Saigon Deli at this intersection but this one is on Jackson just west of twelfthand with $2 pork Banh Mi you cant go wrong, I am going to Indiana for work so I went in and got 6 Pork', 
         'Ive eaten here a few times in the past and thought it was decent. I went last night, however, and our meal was really subpar so the place seems to have gone down hill.We had chow fun noodles and the hollow vegetables with chili sauce']

lines = lines.map(lambda x: preprocess_text(x))

# create vocabulary
vocab_processor = tf.contrib.learn.preprocessing.VocabularyProcessor(MAX_DOCUMENT_LENGTH)
vocab_processor.fit(lines)
with gfile.Open('vocab.tsv', 'wb') as f:
    f.write("{}\n".format(PADWORD))
    for word, index in vocab_processor.vocabulary_._mapping.iteritems():
      f.write("{}\n".format(word))
N_WORDS = len(vocab_processor.vocabulary_)
print '{} words into vocab.tsv'.format(N_WORDS)

# can use the vocabulary to convert words to numbers
table = lookup.index_table_from_file(
  vocabulary_file='vocab.tsv', num_oov_buckets=1, vocab_size=None, default_value=-1)
numbers = table.lookup(tf.constant(lines[0].split()))
with tf.Session() as sess:
  tf.tables_initializer().run()
  print "{} --> {}".format(lines[0], numbers.eval())   

1.2.1
118 words into vocab.tsv
This might be the best "taco" truck on the planet. Hidden between a smoke shop and The Home Depot, this semi permanent, stylish and clean cafe on wheels. --> [ 42  12  41 118  33 119  51  47 118 119  96  39 112  54   1  87 111  56
 119  81  19 119  23  87 117  99  47 119]


In [34]:
!head vocab.tsv

ZYXW
shop
just
cheap
go
Indiana
its
We
seems
laid


Sprawdzenie działania modelu na małym zbiorze danych

In [163]:
%bash
echo "bucket=${BUCKET}"
rm -rf outputdir
export PYTHONPATH=${PYTHONPATH}:${PWD}/trainer
python -m trainer.task \
   --bucket=${BUCKET} \
   --output_dir=outputdir \
   --job-dir=./tmp --train_steps=500

bucket=dswbiznesie
5687 words into gs://dswbiznesie/data/vocab_words
{'review_count': <tf.Tensor 'DecodeCSV:4' shape=(?, 1) dtype=float32>, 'avg_rating': <tf.Tensor 'DecodeCSV:5' shape=(?, 1) dtype=float32>, 'reviews': <tf.Tensor 'DecodeCSV:0' shape=(?, 1) dtype=string>, 'categories': <tf.Tensor 'DecodeCSV:2' shape=(?, 1) dtype=string>, 'zip_code': <tf.Tensor 'DecodeCSV:3' shape=(?, 1) dtype=string>} Tensor("hash_table_Lookup:0", shape=(?, 1), dtype=int64)
Tensor("string_to_index_1_Lookup:0", shape=(?, ?), dtype=int64)
words_sliced=SparseTensor(indices=Tensor("StringSplit:0", shape=(?, 2), dtype=int64), values=Tensor("StringSplit:1", shape=(?,), dtype=string), dense_shape=Tensor("StringSplit:2", shape=(2,), dtype=int64))
words_embed=Tensor("EmbedSequence/embedding_lookup:0", shape=(?, 77838, 10), dtype=float32)
words_conv=Tensor("Squeeze_1:0", shape=(?, 19460), dtype=float32)
embed_categories shape
(?, 100, 10)
categories_out shape
(?, 100, 1)
Tensor("DecodeCSV:5", shape=(?, 1), dtype=

Copying gs://dswbiznesie/data/train.csv...
/ [1 files][  3.2 MiB/  3.2 MiB]                                                
Operation completed over 1 objects/3.2 MiB.                                      
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_num_ps_replicas': 0, '_keep_checkpoint_max': 5, '_task_type': None, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fc359cd3f90>, '_model_dir': 'outputdir/', '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_session_config': None, '_tf_random_seed': None, '_environment': 'local', '_num_worker_replicas': 0, '_task_id': 0, '_save_summary_steps': 100, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_evaluation_master': '', '_master': ''}
Instructions for updating:
Monitors are deprecated. Please use tf.train.SessionRunHook.
INFO:tensorflow:Create CheckpointSaverHook.
2017-11-19 01:06:31

<h2> Trenowanie modelu w Cloud ML </h2>

Jak widać kod pythonowy działa przy bezpośrednim wywołaniu. Można przetestować go lokalnie w Cloud ML.

In [ ]:
%bash
rm -rf outputdir
gcloud ml-engine local train \
   --module-name=trainer.task \
   --package-path=${REPO}/notebooks/trainer \
   -- \
   --bucket=${BUCKET} \
   --output_dir=outputdir \
   --job-dir=./tmp --train_steps=500

5687 words into gs://dswbiznesie/data/vocab_words
{'review_count': <tf.Tensor 'DecodeCSV:4' shape=(?, 1) dtype=float32>, 'avg_rating': <tf.Tensor 'DecodeCSV:5' shape=(?, 1) dtype=float32>, 'reviews': <tf.Tensor 'DecodeCSV:0' shape=(?, 1) dtype=string>, 'categories': <tf.Tensor 'DecodeCSV:2' shape=(?, 1) dtype=string>, 'zip_code': <tf.Tensor 'DecodeCSV:3' shape=(?, 1) dtype=string>} Tensor("hash_table_Lookup:0", shape=(?, 1), dtype=int64)
Tensor("string_to_index_1_Lookup:0", shape=(?, ?), dtype=int64)
words_sliced=SparseTensor(indices=Tensor("StringSplit:0", shape=(?, 2), dtype=int64), values=Tensor("StringSplit:1", shape=(?,), dtype=string), dense_shape=Tensor("StringSplit:2", shape=(2,), dtype=int64))
words_embed=Tensor("EmbedSequence/embedding_lookup:0", shape=(?, 77838, 10), dtype=float32)
words_conv=Tensor("Squeeze_1:0", shape=(?, 19460), dtype=float32)
embed_categories shape
(?, 100, 10)
categories_out shape
(?, 100, 1)
Tensor("DecodeCSV:5", shape=(?, 1), dtype=float32)
Tensor("De

Copying gs://dswbiznesie/data/train.csv...
/ [1 files][  3.2 MiB/  3.2 MiB]                                                
Operation completed over 1 objects/3.2 MiB.                                      
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_num_ps_replicas': 0, '_keep_checkpoint_max': 5, '_task_type': None, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fb244ed1fd0>, '_model_dir': 'outputdir/', '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_session_config': None, '_tf_random_seed': None, '_environment': u'cloud', '_num_worker_replicas': 0, '_task_id': 0, '_save_summary_steps': 100, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_evaluation_master': '', '_master': ''}
Instructions for updating:
Monitors are deprecated. Please use tf.train.SessionRunHook.
INFO:tensorflow:Create CheckpointSaverHook.
2017-11-19 01:49:1

<b> Wizualizacja procesu uczenia w Tensorboard </b>

In [ ]:
from google.datalab.ml import TensorBoard
TensorBoard().start('{}/notebooks/outputdir'.format(REPO))

In [ ]:
from google.datalab.ml import TensorBoard
for pid in TensorBoard.list()['pid']:
  TensorBoard().stop(pid)
  print 'Stopped TensorBoard with pid {}'.format(pid)

Tak przygotowany kod można uruchomić w klastrze Cloud ML. Status joba uczącego znajduje się w konsoli GCP.

In [184]:
%bash
OUTDIR=gs://${BUCKET}/trained_model
JOBNAME=hygiene_$(date -u +%y%m%d_%H%M%S)
echo $OUTDIR $REGION $JOBNAME
gsutil -m rm -rf $OUTDIR
gsutil cp trainer/*.py $OUTDIR
gcloud ml-engine jobs submit training $JOBNAME \
   --region=$REGION \
   --module-name=trainer.task \
   --package-path=$(pwd)/trainer \
   --job-dir=$OUTDIR \
   --staging-bucket=gs://$BUCKET \
   --scale-tier=BASIC --runtime-version=1.2 \
   -- \
   --bucket=${BUCKET} \
   --output_dir=${OUTDIR} \
   --train_steps=5000

gs://dswbiznesie/trained_model europe-west1 hygiene_171119_020803
jobId: hygiene_171119_020803
state: QUEUED


Removing gs://dswbiznesie/trained_model/#1511056973387375...
Removing gs://dswbiznesie/trained_model/events.out.tfevents.1511055884.cmle-training-master-fcdd4d6b12-0-1sbp9#1511056977183360...
Removing gs://dswbiznesie/trained_model/eval/events.out.tfevents.1511057006.cmle-training-master-fcdd4d6b12-0-1sbp9#1511057007314455...
Removing gs://dswbiznesie/trained_model/export/#1511057009699243...
Removing gs://dswbiznesie/trained_model/__init__.py#1511055512255850...
Removing gs://dswbiznesie/trained_model/checkpoint#1511056975729373...
Removing gs://dswbiznesie/trained_model/eval/#1511056427415344...
Removing gs://dswbiznesie/trained_model/export/Servo/#1511057010103836...
Removing gs://dswbiznesie/trained_model/eval/events.out.tfevents.1511056427.cmle-training-master-fcdd4d6b12-0-1sbp9#1511056970694769...
Removing gs://dswbiznesie/trained_model/export/Servo/1511057007/#1511057010552574...
Removing gs://dswbiznesie/trained_model/export/Servo/1511057007/assets/#1511057011175164...
Removing

<h2> Opublikowanie wytrenowanego modelu </h2>

In [182]:
%bash
gsutil ls gs://${BUCKET}/trained_model/export/Servo/

gs://dswbiznesie/trained_model/export/Servo/
gs://dswbiznesie/trained_model/export/Servo/1511057007/


In [192]:
%bash
MODEL_NAME="hygiene"
MODEL_VERSION="v2"
MODEL_LOCATION=$(gsutil ls gs://${BUCKET}/trained_model/export/Servo/ | tail -1)
echo "Deleting and deploying $MODEL_NAME $MODEL_VERSION from $MODEL_LOCATION ... this will take a few minutes"
#gcloud ml-engine versions delete ${MODEL_VERSION} --model ${MODEL_NAME}
#gcloud ml-engine models delete ${MODEL_NAME}
gcloud ml-engine models create ${MODEL_NAME} --regions $REGION
gcloud ml-engine versions create ${MODEL_VERSION} --model ${MODEL_NAME} --origin ${MODEL_LOCATION}

Deleting and deploying hygiene v2 from gs://dswbiznesie/trained_model/export/Servo/1511060051/ ... this will take a few minutes


ERROR: (gcloud.ml-engine.models.create) Resource in project [dswbiznesie] is the subject of a conflict: Field: model.name Error: A model with the same name already exists.
- '@type': type.googleapis.com/google.rpc.BadRequest
  fieldViolations:
  - description: A model with the same name already exists.
    field: model.name
Creating version (this might take a few minutes)......
....................................................................................................done.


<h2> Serwowanie predykcji </h2>

Opublikowany model, zostanie użyty do serwowania predykcji. Aby otrzymać predykcje, wystarczy wykonać JSONowy request do api predykcji.

In [193]:
from googleapiclient import discovery
from oauth2client.client import GoogleCredentials
import json

credentials = GoogleCredentials.get_application_default()
api = discovery.build('ml', 'v1', credentials=credentials) #,
           # discoveryServiceUrl='https://storage.googleapis.com/cloud-ml/discovery/ml_v1beta1_discovery.json')

request_data = {"instances":[
    {
      "reviews": preprocess_text(" Never have I been more insulted from a manager. I would highly recommend NEVER visiting this cafe. This man yelled at me on 3 different occasions during my afternoon attempts to study.I purchased coffee and a bagel. First I was moved to a smaller table because \"people who were actually eating a meal\" needed to sit at my seat. Apparently your portion determines your table hierarchy.When I was sitting at a table with other empty seats awaiting a study group I was literally yelled at by the manager a second for taking up too much room. I politely explained anyone was welcome to sit here, but would need to make room when my group arrived. Then when my group was situated - he kicked out someone else, and forcibly moved us to a smaller table. And an hour later said we had to leave in 15 if no one would buy anything else. Like he would ever get a cent more of my money.If you enjoy getting forcibly reseated every hour, like unreliable wifi and find it pleasant to have a jerk manager yelling at you for being a patron -- then I highly recommend checking this place out. Otherwise even Starbucks is better. Needing some lunch on a day I hadn't even had breakfast, I went up to the Cafe on the Ave on a Sunday when I was at work at UW. They had a decently large selection. I was looking for something vegetarian, so I got the Mediterranean omelette with whole wheat toast and hash browns and coffee.What I ordered for coffee, what was listed on the menu, and what they called out when my coffee was ready was not the same. It being after 1 pm (and having been up since 7 am without anything to eat) having different names for the same thing was not helpful for me. They have a strange, call out your order number and you come get it and bus your own table thing going on for the cost and decor and set-up of the cafe. The food coming up was slow. Like 45 minutes. That's pretty bad. It was past lunch rush, so there's no reason for an omelette filled with vegetables to take that long. And then have the wrong kind of bread on it. And they put way too much butter on the bread, so I couldn't eat it. The omelette was greasy. How do you even make a greasy omelette? The coffee was good. The guy serving the coffee and coffee drinks was friendly. The rest of the staff was stand-offish. The rest of the customers sat like they owned the place and therefore getting anywhere in the cafe was a difficult maze of a walk. Overall, the place was okay. I won't make any effort to go back. I'd like to like it, but it's just not worth the time and cost. If it weren't for its location and size, I don't know how this business would thrive. I've had really good service a few times, really bad some other times. It almost always takes forever for your food to get to you. I remember it took them like 15 mins to get my hummus plate to me which was amazing because I can't imagine it takes that long to plop some hummus on a plate and slice up some things to dip in it. The food is mediocre at best. If you take into account how much it costs for what you get, it leaves a worse taste in your mouth. The soups are often lukewarm. The sandwiches and burgers are okay. Breakfast isn't anything special. Their coffees and teas are a-okay. The only thing I like from here are their fries -- they're actually pretty tasty.Then again, I could say a million of things to express how I'm far from impressed about everything here but I will probably be here in the next couple of weeks or so for a group meeting or to meet up with a friend or to study. It's just too conveniently located and there are lots of outlets and the wi-fi works pretty consistently. Mediocre, yet overpriced relative to quality, salads &amp; coffee-- I wish I had checked Yelp before coming here!Well, never again. Too many other places to try! I came in here and politely ordered the banana with yogurt. I asked if I could add granola, offering to pay for the extra item. The guy at the counter, with brown hair and glasses, flat out told me \"No, we can't do that. You have to get the other one.\" The other one had berries, and I didn't want berries. He told me again, that there was no way for me to get granola added to my dish (WHICH IS EXACTLY THE SAME THING ONLY WITHOUT BERRIES)... um, OK. Totally rude.It's not that I was asking for something extravagant, luxurious or even unreasonable...it's GRANOLA. The coffee smells and tastes burnt, and the internet was so bad that I had to connect to Starbucks' internet. Not worth high prices and snobby (lack of) service. Oh terrible, terrible place. &#160;Manager, manager's wife, and employees were constantly hovering over us, trying to get us to order as quickly as possible. &#160;I wasn't sure why there was a big rush; there was plenty of space in the restaurant.After the third round of obnoxious hovering, our group finally ordered. &#160;Mediocre food. &#160;Bleh. I came in only because I forgot my breakfast, which is good, because it's an excellent reminder to NEVER EVER FORGET YOUR BREAKFAST if it means you'll be coming here instead. I don't eat bread, and I don't eat potato - so, I ordered the standard eggs-meat-hash-toast breakfast sans the hash and toast. I was told I'd have to pay for them anyway. I understand, being someone who purchases her own groceries, that potatoes and slices of bread do cost less to buy (and therefore command a lesser price to the customer) than eggs and strips of bacon, but that would mean that the order would ring up to about $10 for a couple eggs and however many pieces of bacon they decide to give you with an order. That seems a bit... off. When I asked if there was a better way to order it so I wasn't, you know, paying $10 for two eggs and (what ended up being three) strips of bacon... I was told (by the obligatory disaffected counter dude) that the sides are priced specifically to make stuff ring up with a similar amount. Now, I respect a restaurant's need to prevent creatively-broke-college-student menu-hax from putting them out of business, but REALLY? Next time, I'm just eating lunch early. With all the square footage, and the raised outdoor seating, it's a shame to me that this place has such inflated prices on the food. Prices aren't listed on the menu online, but I recall coming here and ordering a sandwich -- not a hefty deli sandwich mind you but a meager portion seemingly for someone watching their weight -- and paying around eight dollars. It's not an outrageous sum and the ingredients were certainly high quality but, I don't know, if there was some new aged pricing strategy in place where you 'pay what you want to,' like Radiohead used with 'In Rainbows\" back in 2007 I would not have forked over eight dollars for that sandwich. I would have given them six dollars, and that would have been fair. This isn't somewhere I'm pining to go back to, but I don't object when a friend who really likes the local art on the wall, or the board games in the back, or the black on blacker than black paint really wants to get a drip coffee and scones from somewhere other than the Starbucks across the street. A good to place to study, and the wireless is reliable, which is more than can be said for some other coffee places in the area. &#160;The food is consistently good, and the coffee is delicious. &#160;I usually order the breakfast burrito, and it never fails to satisfy. &#160;The fries are not fantastic, a little too McDonald's-like for my tastes. Why the average rating? &#160;The waitstaff is what you would expect from part-time college students: inattentive and a little too \"chill\" for my tastes. &#160;Seems like they are all standing around texting or changing the PA music. Is it a bar? a cafe? a restaurant?....it's a little bit of everything...Came in this morning after my early class to grab a quick bite to eat. I've heard good things about this place and how fresh the breakfast was so I was definately looking forward to trying it out. I ordered \"the ave\" omelette which came with a three egg omlette stuffed with a &#160;handfull of chopped bellpeppers, onions and diced sausage. There was a descent side of hash and a piece of wheat toast sliced into two. All this for about....close to 11 bucks after tax.....Overall it was a good meal, the portion was enough to fill me up but wasn't anything near satisfying. Service was kind of dull...wouldn't hurt to smile at customers once in a while. The food was really fresh and delicious...and although the hash was a bit greasy I still liked my food overall. I wanted to give four stars for that alone, but really.....10 bucks for a 3 eggs omelette, a slice of toast and some hash? The profit margin is like....7 bucks. I would price it at 7 or 8 bucks max. If you're going to overprice your food at least do it with good service that would put a smile on people's face. This is a cafe...not a high school cafeteria. Bad coffee, greasy hashbrowns, and uninspired eggs; too much money for the quality. Am I paying for the fast wifi? Because it is pretty alright here... that gives them an extra star.They left a spot on the receipt for a tip. What for, busing my own table?There are better places that don't have to put \"On the Ave\" in their name to be truly local. What a great place to study!Went there a couple weeks ago for the first time (in awhile) on a Sunday night around 10pm . There was a plenty of room to sit, and I was able to find an outlet too. I guess its open till 12am everyday? I ordered a 12oz vanilla latte, which was not too bad. I think it really dependson the barista, of course, and I'd have to say that I've had better. Of all the drinks I've gotten there, I think the caramel macchiato was my favorite :)\"Free wi-fi\" when you order something - the password is usually printed out on the receipt. Terrible service. I had to get my own silverware and condiments. I would never go back here. This is my second time to Cafe on the Ave. When we visit Seattle, we stay at the College Inn and this place is a good walk from there. I could easily go to Starbucks, which is closer, but I like supporting the little guys, ya know?I ordered a 12 ounce soy cap with an extra shot of espresso (my old standby and the drink I have painlessly enjoyed everyday for almost 15 years). The gal at the counter started at me blankly and said, \"do you mean a 16 ounce soy latte?\" Um, no? Last time i checked those were two totally different drinks. I replied with, again, what I wanted. Again, blank stare. What I got was a 16 ounce soy latte. Not wanting to argue, I sat at a bistro table and drank it. It was ok- kind of lukewarm, borderline cold.Not sure what the snobbery/'tude is all about. Sure the space is big and has decent artwork on the walls, but it is hardly unique. I think I will check out Slapshot coffee next time, a block up the street. Would have given ZERO starts if that was possible. I have a high tolerance for high poor customer service but this place simply left me in shock today. I had been to Cafe on the Ave a few times in the past, and their food/drinks are actually not bad. Previous experiences there were neutral, but today's left me with the \" I AM NEVER STEPPING MY FEET INTO THIS PLACE AGAIN\" feeling. &#160;They hover over you until you order. &#160;Until then I was fine. Not a problem until one of their employees essentially threw a menu at me while the other one yelled that if I didn't order within 5 minutes, I would have to leave the establishment. Yes, a business located in the U district is at the risk of getting customers who just sit there and read/study and don't place an order. I understand how that could be stressful for business owners, but I made a point that I was not that type of customer and that I was going to order soon. Still, I was yelled at and embarrassed in front of the the whole coffee shop. And there is more: When I was telling another employee that his coworker had basically thrown a menu on my face, the barista who had been rude earlier stepped in, denied, and emphasized that if I wanted to, she could \"show me what throwing a menu on my face would look like\". Completely uncalled for, and very disrespectful. &#160;To be fair, one very nice employee actually apologized for his co-worker's behavior. NEVER AGAIN reading the other reviews on here, i guess i'm not alone in experiencing terrible customer service. the food is actually decent, 3 stars, probably, but my experience with a server who was old enough that he seemed to be a manager made me say to myself that i'd never step inside this joint again.in a neighborhood with a dozen cafes why would i come here again? oh yeah, i guess i won't The place was really loud and dirty so it was hard to study. Plus it is always crowded so its rare to even get a spot in there. The food is simple and decent. But way over priced for what it is. For a simple and small breakfast with eggs, toast, and hash browns it will cost you well over 10 dollars. The service is always cranky every time I've been there. I recommend going down the street and going to another place to eat or study. I had breakfast there. Pancakes and milk cost me a little over $8. I didn't have to pay a tip since you pay up front which was nice. But anyway I got two really meager pancakes. Had they given me 250% of what I gor that would have been almost a decent amount. It was really a laughably meager breakfast. As soon as they brought it out I was like oh jeeze. It took me about sixty seconds, or five or six forkfuls I think, and I was out of there. I think it literally took me a minute and I'm not bragging about how fast I can scarf down pancakes here.I know this isn't really a happening place for breakfast, and this may not be the best criteria to on which to judge this place, but for the eight bucks I could have just gotten pancake mix, some cheap syrup and a whole gallon of milk, ya know? This was a really lousy breakfast. Don't go here for breakfast."),
      "categories":"Breakfast & Brunch,Cafes,Restaurants",
      "zip_code":"98105",
      "review_count":18,
      "avg_rating":1.8421052631578947
       },
    {
      "reviews": preprocess_text(" I am so sad to have to slash my review from five stars to two, but it has to be. &#160;When I first discovered this place three years ago I fell in love with it. &#160;It was a one-man operation. &#160;The owner was both your very genial server and the cook. &#160;The food was good and obviously prepared with care, and the atmosphere was right out of Paris, laid back and intimate.Over the last few years, word obviously spread about the Thomas St. Bistro. &#160;Each time I returned, it was a bit more crowed, and the owner was running a bit more. &#160;About two years ago a server appeared, and the owner was less present, and shortly after that a guitar player also appeared. &#160;I still loved this place because the food was good and the atmosphere was still super-romantic and European.Then I went back last night, and was horrified. &#160;The first question I (and every guest) gets asked is 'are you using a coupon tonight.' &#160;Not too romantic. &#160;The place is packed, and there is a line of people out the door. &#160;Too many reservations, not enough space. &#160;Turns out that what used to be a great little restaurant, tucked away off the beaten path, has turned itself into a massive groupon/travel zoo/living social machine, churning the bargain-seekers through as fast as possible. &#160;The food is not nearly what it used to be, either in flavor or presentation, and the atmosphere is not relaxing at all. &#160;I don't know if I will ever go back. &#160;I certainly won't go back until well after the groupon coupons have run their course. &#160;I guess it's a good thing for the owner to have so much business, but I doubt that he's going to be getting much repeat business anymore. &#160;It's very sad. I've walked by this place for a couple years, wondering what it was like inside (the food, ambiance, etc). &#160;It looked romantic. &#160;I put it on my list of places to check out.I finally went in last Saturday and I wasn't that impressed with the overall experience.I went up to the door, hesitated, turned around to leave and was very promptly greeted by a very friendly waiter who opened the door. &#160;We chatted about the place and I decided to come in. &#160;The actual restaurant is adorable (very small, cozy, felt European). &#160;The food was only okay (pureed vegetable soup, bread and cheese, and pasta with feta and olives). &#160;Things got a little weird when I thought that dessert was included with the three course meal, only to find out that the two tiny pieces of bread and three tiny pieces of cheese was the first course (followed by the soup and the pasta). &#160;They ended up bringing me a dessert anyway, after basically encouraging me to hurry up because there were people waiting outside for a table. &#160;The dessert was not good (some kind of warm banana thing with raisins).Not a horrible experience, but it fell way below my expectations. &#160;Not a bad place, but also not great and the service wasn't consistent (I initially felt very welcome, but then felt like I was being asked to hurry up and leave). CREEPY. My 2nd attempt at eating at this little weird place and I am asking Groupon for a refund.Time #1: We called and called to try to get a reservation. NO ONE ANSWERED THE PHONE. EVER. We got to the restaurant at 9pm on a Saturday and they told us the restaurant was closed at 9pm. You're kidding, right? A new restaurant closing at 9pm ON A SATURDAY? Time #2: After calling and calling again to try to get a reservation, we tried once more to just show up. My girlfriend sat in the car while I ran up to see if I needed a reservation. The owner asked if I was would be eating with them tonight. Me: \"Yes, please.\" Owner: Well, we are full as you can see.\" I turned and saw three empty tables inside which he explained it was much too hot inside. I checked. He was telling the truth... It was nearly 85 degrees or more INSIDE. So I asked if I could make a reservation for later in the evening and he stares at me blankly. And stares and stares. I figured maybe a language barrier was to blame so I asked again \"Sir, can we get a table later? Blank weird stare. \"Sir, can we eat here or not?\" and he responds, \"Well, as you can see we are full, if you'd like to brave the inside you can.\" It was like the most surreal SNL-like skit ever. He stared like he either couldn't understand me or had fallen deeply in love with me. &#160;I seriously got the creeps. With this kind of horrible experience just trying to get seated, I can't fathom what the experience would be like with that dude hovering over our table on the patio.If you bought a coupon to eat there, either act now or get a refund. This place will be gone in less than 6 months. this was definitely a unique experience ...to say the least. After looking for parking for 20 minutes (non existent) we came in to find the smallest place i have ever been to...filled with old chairs with dirty cushions - good thing there were plenty of places to sit since it was empty. We were greeted with \"do you have a coupon?\" - which honestly just made us laugh the whole night. &#160;All the plates don't match, the main dish on the menu (if you even want to call 6 items a menu) was not available ... the salad was a bunch of weeds thrown together with hidden valley dressing and two small tomatoes. Entrees were so bland and i could not help but to think one of the dishes tasted identical to a frozen costco ravioli pack i have tried. i wouldn't put it past them. The only dessert that was offered to us was...drumroll please... VANILLA ICE CREAM. thats it. just vanilla ice cream. all in all this place was so horrible that it made us laugh and try to forget we were celebrating a special occasion. that will be the last time i go somewhere without researching it because based on other reviews it sounds like i was not the only one.I will mention, i thought the waiter was very nice and seemed to be trying with the few resources (and experience) that he had.I also thought the food was ridiculously overpriced for what it was, location, setting and of course taste. I think they just increased them so they can continue to run these coupon deals so people think they are getting a great discount, when in reality, this entire meal should cost less than 20 dollars. I cannot believe that this restaurant gets any good reviews as all. &#160;I was amazed at just how amateur a run place this is and what deceptive advertising! &#160;I advise everybody to read the reviews before deciding to go. &#160;Unfortunately I only started reading the reviews while in the car coming to the restaurant and then I had a sinking feeling. &#160;That feeling didn't improve. &#160;Our waiter was very sweet and helpful, no problem with the service but the food was something I could do on an off day, the portions were absolutely miniature and not particularly well cooked at all. &#160;This place was a rip-off no matter the amount of the coupon - I'm sorry I wasted my money on it. &#160;My husband who doesn't drink wine couldn't get anything but tepid water that was in the table jugs. &#160;No bread available. &#160;They charged us $18 for 4 tiny glasses of wine even though it was supposed to be part of the special we purchased through Travelzoo. &#160;I paid it anyways, I guess I felt sorry for them and considered it a donation.The decor is another issue, it's just like someone's cluttered home, stained cushions, mis-matched tableware and all. &#160;Just sad. &#160;I didn't feel good in this space at all, just very uncomfortable and couldn't wait to get out. I have spent $50 on this place and haven't even been there. &#160;Why? &#160;Because I bought a Groupon I now have absolutely no plans on using.What a terrible experience - before I even stepped foot into this place.The owners here obviously have zero pride in their establishment. &#160;The website is a damn mess - they didn't even spell check their menu! &#160;Hey, Thomas Street Bistro - DESSERT is a yummy after-meal treat, DESERT is a dry or barren area of land.So I figure, whatever - maybe it's a small place with no spelling or grammar skills and without any concept of fair pricing or the money to hire a web developer... I bought the Groupon on a whim so I'll use it anyway.I called 4 times before getting a hold of someone to make a reservation - just to find out they were closed on the day I wanted to go (a Tuesday). &#160;After the 2nd call, I tried emailing... and actually did receive a response several hours later.In my email I requested the reservation time and day (again, a Tuesday) and asked a couple questions about their menu and the opportunity to use the Groupon (sometimes places don't allow you to use it for dinner or on the weekends, etc.)The response: \"closed on Tuesdays\"How professional is that? &#160;No offer to make a reservation on a different day? &#160;Don't even bother answering my questions?Thanks. &#160;I'll DESERT any intention I had of going here - ever - and hope that Groupon will honor their return policy.BTW - IF you went here with a Groupon and hated it, check out Groupon's return policy. &#160;I have heard good things about it. I have eaten at this place 3 times, and it has gotten progressively worse. I live in the neighborhood and want this place to be a go-to for a quick meal in the 'hood -- but to be honest the service is abhorrent and the place is too small to even pretend to be a neighborhood bistro. Having lived in Europe I have dined in tiny bistros -- but this place is a closet and the kitchen is more of a crawl space. The last and final time I tried the restaurant it was a Tuesday night -- only about 3 other tables, the service was once again slow slow slow -- and he didn't have 1 of the 3 items on the menu -- which was chicken breast -- and the only other protein option was a pork chop. I chose the ravioli -- which was a horrible mistake. The fact that the owner has begun advertising on every Groupon type of website -- and then openly treats people with coupons with disregard is a testimony to their awful service. Two of my good friends that eat at the bistro regularly often have heard him openly complaining about Groupon customers -- which in itself is inexcusable. If you don't want business, don't offer yourself on the site. The owner should try and focus more on what he can do well -- and that isn't a 3 course french style prix fixe menu -- and brush up on customer service! My second time here. Can't remember what i had the first time but i remember it being better. Had a buywithme deal, so a standard 4 courses plus a bottle of wine. At least the wine was good....-Vegetable soup to start: was surprisingly yummy. Sadly, the best part of the meal-\"salad\" came as a little slice of a roll, a couple cheese crumbs and a quarter piece of prosciutto - are you effin' kidding me?!?!-I had the rigotoni and bf had the salmon spaghetti - both were mediocre.-Dessert was a few slices of bananas thrown into a cup with random shit from the kitchen mixed in. DISGUSTING!! Couldn't tell if it was a raisin or bugs.... maybe some coconut flakes (like 5) and some weird semi-sweet sauce. I understand that you are a small business, but if you can't afford it, you shouldn't keep doing these groupon deals. I had a great experience the first time, which is why I came back a second time and brought friends. But this second experience was AWFUL and I will never come back. Went for dinner and even though the restaurant wasn't full or very busy, it took a long time to get seated and two parties that were seated after us got their menus and had their orders taken before we even got our menus.Food was OK - not terrible, but not great, certainly not good enough to make up for the service issuesDisappointing because it's a cute, cozy place in the neighborhood, but it's unlikely we'll be heading back Went to dinner here to use our TravelZoo coupon, and it led to a lively discussion of whether or not this restaurant is perpetuating fraud. &#160;Do avoid at all costs. &#160;Among the highlights of the night: a water glass with a yellow stain on the bottom, and when we asked for a new one, we received a condescending \"Oh, that sucks, doesn't it?\" from the waiter, who was well aware of how bad the restaurant was, and seemed to enjoy it. &#160;The soup was \"vegetarian,\" despite being chicken broth-based. &#160;Upon ordering the apricot couscous dish, we were told that they were out, despite the fact that it was one of four items on the menu (one of which appeared to be low-rent spaghetti). &#160;For dessert, we waited at the table for a good 15 minutes with empty glasses while the waiter blatantly ignored us, before he brought us our two slices of Safeway-brand cheesecake. &#160;What an evening!After much discussion, we concluded that this place is blatantly scamming the public and will be gone in a few months. &#160;No restaurant can provide such low quality food and service and expect to last. &#160;If you have bought a coupon, eat your losses and go elsewhere; the $8 in tax you'll have to pay will be better spent at Taco Bell than this godforsaken place. We gave them another try with brunch. They did not disappoint me to disappoint me again. Seemingly interesting food items, but they all tasted rather boring. I see four friendly servers--possibly one of them was cook --for only two seated tables, including us, at the time, but they were too busy setting up the place to give each table enough/fast services. Too bad, otherwise, it is a cozy little place.I don't think I will go back there anymore.Oh, BTW, they said \"three-course\" brunch everywhere, but actually only two-course. No dessert as they used to claim. The same price though. This is a great little space with lots of potential. Tiny restaurant and outdoor seating. Could be a great neighborhood restaurant.Unfortunately, they've got a lot of work to do to make it good, yet alone great.The three course brunch has become a two course brunch and there's a charge for the average at best coffee. If you take it sweetened, use the raw sugar because they don't offer any alternative sweeteners. The Mediterranean omelet was undercooked and runny inside. It was rather bland tasting and when I asked for salt was told that they were a sodium free restaurant. Now, I'm all for being healthy, but c'mon let that be my choice, not yours. If you don't want to cook with salt, fine but I'd like the option of adding it if I prefer.There were a couple of tables filled on the patio and I was the only person inside the restaurant. The server hovered outside the kitchen which was about 10 feet from the entrance and me. He stood there looking out and I felt as if I was being watched. Kind of hard to \"sit long, talk much, laugh often\" as is their tagline. To their credit, they did give me a coupon for a complimentary \"3\"course brunch. I won't be using it but will pass it on to someone else. Maybe they'll have a better experience.I hope they take a look at the comments here. I really would like for them to succeed and think they have the potential. They just aren't living up to it. Worst place I've ever Been ... Rude waiter treated us very wrong .. I would never go there or recomend &#160;it to anyone gave us a bottle of water to share among 6 of us . Ordered drinks gave us half filled wine glasses of soda .Our friends had groupon didn't even give them drinks, we left I can't even give it a whole star We had the opportunity to dine here for the first time this weekend. It's been 2 years that I've walked by here, hoping some day to try out this cute, cozy little place. The only thing that was holding me back; a lack of vegetarian options. A \"soon to expire\" Living Social coupon forced the decision though and we gave it a try.How sad it was to be so completely disappointed in this place.A few things of note. 1) Menu: We'd looked on line at their menu and saw a couple veggie options that sounded good. However once we got there the menu was different. That's not &#160;uncommon nor a terrible thing, but then the waiter went on to say that the ONE vegetarian option on the current menu was not available either. He did point out that they had two choices not on the menu which we ended up trying.2) Service: It was awful. One waiter for the entire restaurant. It's small, but not that small. Everyone around us was unhappy and had issues. It wasn't as if he wasn't trying. He was, and he was super nice. He was just terrible. He forgot orders, forgot to take orders, didn't bring water, utensils or napkins to people...at least we got free soup out of the deal (only good food we had), but seriously, this is a fairly upscale, expensive restaurant. Let's get some service here.3) Food: Pretty mediocre. None of it bad, only one thing good. We had the 3 course meal (4 courses, because of bad service we ended up with free soup) which started with a salad. Pale, browned ice berg lettuce with a little feta and one slice of tomato. I can't remember the last time I've seen ice berg in a Seattle restaurant. Inexcusable. Next we had soup (the noted free course). It was really quite good and it's good we got the free course since it was the only thing worth while we ate that night. It was a yummy carrot / squash cream soup, very good. Then the main course; Pasta with veggies (me) and my GF had ravioli. The ravioli was salt and bland. Something reminiscent of mall food-court food. The pasta was pretty bad. I'm pretty sure this is what pasta in a can tastes like. There really is no excuse. If you are paying $50 - 60 for dinner you should expect better quality. &#160;Granted that this is probably a meat place and maybe it is unfair to judge them by pasta, but ya know, if you can't make it good, don't serve it.The last course was desert which is a cream puff like one would get at a pho place. it was OK, but I wouldn't call it a course on it's own. I had high hopes for this place, unfortunately my fantasy was shattered. it will be my last visit there..... See Clare K 7/8/2011 review. We had the exact same experience, except our coupon cost $50 for a 4 course meal (supposedly valued at $115). Include tax and we had 2 more glasses of wine, we ended up paying more than $80 for something that should have cost $40. All the entrees were $15 and came with soup and salad anyway. NO NO NO. My next stop this evening: Travelzoo, to let them know that this place is giving their deals a bad name. Very quaint, cozy and has a nice atmosphere. &#160; Good for a first date. Just had the worst meal out I've had in a long time at Thomas Street Bistro. Terrible Service serving Airplane Food in a Lousy Atmosphere. And expensive to boot. So sad.Add to that a rude owner and truly...terrible, terrible food and I will not be going back.Tant pis. its an \"Experience\" sit and wait. we went to brunch.. sat for 15 mins before we found out that it was a fixed menu. it sounded great on paper.. however the items didn't come out how it was written.. cous cous was described to have walnuts.. instead came out with berries.. I'm not complaining about berries.. but i do like to know what I'm about to eat! this \"three course\" meal then had a egg option.. with choice of cheese...when we asked if we could put in our request for the second course we were told it was too soon.. and they were out of eggs! &#160; after our eggs were serves then our beverage came out. &#160;no cheese was present in the meal... I'm not sure what the third course was??just a bizarre experience. we were there for about an hour and a half... and the service could have been half the time.. i felt bad for the people waiting. for the sub-par meal and the even longer experience they were going to endure. i can say that the food was good, we ate every bite. but the service was sooooo long and drawn out without a check in that i don't ever see myself going back.. and i wouldn't recommend it unless you had time to waste. What a surprise, the place gets three good reviews within a day of it running on Groupon again. &#160;Coincidence? &#160;Hmm...I hate to sound like I'm piling on, but I have to agree with the overwhelmingly common opinion that this place sucks.The owner has dug himself a hole, the proverbial \"deal with the Devil\" (Groupon) to pull in business. &#160;Make no mistake Groupon is the anti-Christ as far as restaurants go, and if you have to use them (or similar) more than once, it's not a good sign. &#160;Especially if everyone's experience is bad and they never come back.Our disappointing odyssey started when we noticed (like everyone) that the menu offered was absolutely nothing like the menu online-- which looks modestly promising. &#160;Instead, there's lots of cheap and dumbed-down fodder that's nonetheless ridiculously overpriced for what you get. &#160;The \"3 course meal\" they say is worth $20-30 is worth about $10 or $12, tops. &#160;And that's being charitable.We started with a salad which was marginally acceptable. &#160;Next came some soup (don't recall what) which was actually the best part of the meal. &#160;Since the chicken breast and capers &#160;we both counted on was predictably not offered that night (ditto the lamb), we both settled on chicken risotto. &#160;It was tasty, albeit a little salty, and lacking in the hinted at chicken. &#160;And let's be real, here-- it's a plate of rice. &#160;It wasn't $17 worth of food. &#160;It was a plate of ****ing rice with a few chunks of meat. &#160;The dessert that followed was pedestrian. &#160;I can't even remember what it was, except that I remember noting \"I've never personally been able to slice cake this thinly\". &#160;It was all barely \"OK\" for cheap food, but it sure wasn't a $20-30 meal.Throughout this, I have to give credit to the waiter who was pleasant and accommodating, in spite of knowing we (like everyone else) were using Groupons. &#160;We didn't take it out on him, we tipped him well. &#160;It's not his fault.The owner needs to buy a clue: &#160;The Groupon thing (and Living Social, and Daily Deals, and.....et al) won't save your place. Cut bait on that shit-- you're digging your grave deeper and deeper. &#160;Netting 25% of the face value of the Groupon won't keep you afloat even if you trash-down your menu with fodder and inflate the prices to adjust for it. &#160;Groupon people are notoriously cheap anyway, and the few that would come back as full-paying customers (like me) won't do it if the food they get on their Groupon blows, duh? &#160;Sooner or later you'll have to survive on the merits of your food, not on a \"deal\", and price it fairly. &#160;Eventually you'll have to face the music. &#160;Groupon is like selling your soul, and everyone knows it.I feel bad for the owner, cuz the place is cute, the concept is good, and it could be a rockin' place. &#160;But everyone's wise to his game once they've eaten there once. &#160;Unless he wises up soon, the place is doomed to an untimely Groupon-fueled death. This is Sad. They are throwing themselves on all the deals' websites and stating that you're getting half off. It's almost laughable. Their average reviews used to be over 4 stars and now it is averaging to two. SAD, RIGHT? If I had to choose, I'd rather go to Spaghetti Factory where I don't need a coupon and know I am at least getting served what is stated on the menu, not some surprise dish that was 'described incorrectly on the menu. Fix that then! This place is confusing. Nice try. They are better off if they lower their pricing and stop selling these groupons. I swear I've seen them on every site. That is why that is the first thing they ask you when you eat there. \"Are you using a coupon today?\" &#160;WTF!? My experience here sucked if you read my first review. I was the poor girl that had to sit behind the door. I went here recently and thought the food was just okay. &#160;We were using a certificate for our meal, and it seems like most people going there these days must be, because the first thing out of the waiter's mouth, was, \"are you using a certificate tonight?\" The menu was much smaller than what is on the website. &#160;My biggest complaint in offering groupon-like certificates is regarding the merchant's desire to shortchange the guest, rather than himself. &#160;He could think one of two ways: &#160;I will lose money or come out even on these guests, but the food will be so amazing, that they'll want to come back and pay full price, OR, he could say, these guests will probably not come back, therefore I should provide a menu that allows me, the merchant, to make money on each meal. &#160;So, the menu offered a very small amount of entrees, which seem like he could make a profit on (pasta, risotto, salad) and cook up pretty easily. The soup that was given to us was delicious, and the dessert afterwards was just okay. &#160;I think if he changes his philosophy on using these certificates, he can generate a lot of new customers rather than one-time guests. &#160;Not going back. I will preface that this review is several months old, I am still able to recall both the highlights and disappointments of my meal here. &#160;My husband and I moved to the city of Seattle within this last year, and I had been perusing several different food websites for places to eat in our new neighborhood. &#160;I had seen great reviews on the Thomas Street Bistro on Yelp and other restaurant review websites and blogs (again, this was several months before, and there were hardly any 1 or 2 star reviews, mainly 4 and 5 stars at the time), that I was super excited when I saw a Groupon being offered for the restaurant. &#160; I snatched up a Groupon, and made reservations for my husband's birthday, which was a few weeks after the Groupon was offered. I was excited, and thrilled to be trying out this cozy, little, restaurant, and was hoping to make his birthday special and memorable. &#160;Special and memorable it was, only not in the good sense.When I made the reservations, I let the person know I would be using a Groupon. &#160;Similarly, when I showed up for our reservation, I notified our waiter (who didn't even look up my name or reservation) that I would be using a Groupon. &#160;This did not seem to have an effect on the quality of our service, but service was ....awkward..overall. &#160;Our waiter, overall was nice, attentive, courteous. &#160;There was also another gentleman there, (an owner or manager perhaps?) who was a little bit on the creepy, overly-friendly, on the point of being rude almost. &#160;More details on that later. &#160;We were seated, given menus, and told about the specials of the day. The menu was slightly different than what was listed online on the website, but I was excited to try something different. &#160;My husband ordered their lamb special, and I ordered their chicken and pasta dish. &#160;We were then served soup, which we had not ordered, but was told was included as a part of our meal. &#160;I was excited to have scored an appetizer, as as I would learn later, dessert as well. &#160;Things were looking good!The soup was a vegetable puree, which was tasty, and had a comfort-food sort of consistency. &#160;I thoroughly enjoyed this portion of our meal, and looking back, it was the best part of our meal. &#160;At this point, our server came out and notified us that the lamb special would not be ready for at least 30 minutes, as their chef has just started preparing the meat. &#160;.....wierd..... &#160;So my husband was asked to make another selection from the menu. &#160;I cannot recall what he ended up ordering, but I do remember him being very under-whelemed. &#160; &#160;My chicken and pasta dish arrived and I was shocked... &#160;What arrived on my plate was a piece of chicken breast, that looked identical to the frozen chicken breast that I purchase at Costco, on a bed of pasta, with a thinned out version of alfredo/cream sauce on top, topped with some slices of bell pepper, c and bits of crumbled feta. &#160;The food was meh.... &#160;Mediocre at best. &#160;My husband begrudgingly ate his meal, either a ravioli or salmon pasta, and I chocked down my dish. &#160;The flavors just didn't go together, and the chicken wasn't even seasoned. &#160;It was totally bland. &#160;I had to eat each part of my meal separately order to choke it down because none of the flavors complimented each other and was inedible otherwise. The whole time I was eating my meal, I couldn't get over the fact that they were blatantly serving me a Costco chicken breast that I would have done a better job preparing myself. For dessert, we were each given a slice of red velvet cake. &#160;It was tasty, and sweet, &#160;nice touch. &#160;But again, I had thought I had seen a special on red velvet cakes at Costco the week before, and the cake slice on our table looked strikingly similar to the cake I had seen at Costco the weekend before. &#160;Additionally, I started noticing some inconsistencies. &#160;While we were served a slice of cake each, another table for two was only given one slice of cake to share. &#160;While at another table, the other waiter, the awkward guy, was trying to serve a diabetic man, a slice of cake. &#160;When the male patron refused, the waiter tried to give him some chocolates, and would not leave the poor man alone until he took the chocolates from him. &#160;He was almost rude and pushy about it, and didn't really offer the patron another option. We waited around for our check, and when it was time to pay, we presented our Groupon and the balance in cash. &#160;The other gentleman, the awkward one, seemed to just pocket the cash, and sent us on our merry way. &#160;Two days later, we received our coupon circular from Costco and saw &#160;the red velvet cake we were served. &#160;Identical right down to the piped frosting on the corners of the cake. &#160;In retrospect, I feel totally taken advantage of, and I get more and more upset the more I see the coupons and advertised specials for this place. &#160;Please take your hard-earned money to an establishment where you will enjoy a quality meal, because you won't find that here. No. Don't go. 1 star for food and service. 4 stars for ambiance. The place has so much potential. The location is great, the settings picture perfect, but the food was the only, MAJOR, issue. I had the pasta and meatballs, and the meatballs were from Costco, and the pasta was like jelly. Service was good, but that shouldn't be a restaurants saving grace. Well I read the one stars fearing the worst. &#160;It wasn't that bad but was less than delicious. Tight seating, meager portions, hot venue, decent staff. If its my call I won't be going back. Too many better places to dine. So, I saw the signs on Broadway advertising the 3-course weekend brunch for $7.99, and HAD to see what it was about. I'd walked by this cute, tiny place a number of times and had vowed I'd dine there sometime and here was my opportunity. The boyfriend and I went on a Sunday morning, and were greeted warmly and offered a table. Well, the tabletop directly next to it (by inches) was occupied at the time by a handsome young gentleman playing the Spanish guitar... if I had taken the seat offered, my face would have been in his armpit, hindering greatly his ability to play the music, not to mention making ingesting any food extremely difficult on my part. We chose a different table, and it was so close to the one that the couple next to us was sitting at that it seemed as though we were dining WITH them, which we weren't and had no desire to. I felt so claustrophobic and awkward I couldn't even have a conversation with my boyfriend... but I guess that's my problem, really. Either way, the place is TINY TINY. The first course was a croissant, which was fine, nothing special. For the second course you have a choice of a meat omelet or a veggie omelet, so I got one and boyfriend got the other. I was NOT impressed... the omelet was just meh; kinda bland and served on a bed of iceberg lettuce chunks. \"Bed\" would suggest that it was placed neatly on the plate but actually it looked like the lettuce chunks were just tossed on haphazardly. Dessert was a slice of peach (the server \"believed\" it was peach) cheesecake, which was okay. All in all, the restaurant was too small and the brunch was a meal that made me wish I had stayed in and made my own breakfast. On the upside, the owner was extremely warm and friendly, the service was good and warm, it was nice to have live music, and if I had wanted to make some new friends I wouldn't have had to go far. Darn, I was really hoping I would like this place. Nothing was bad but then again nothing was good.There are charms to how cozy this place is, how mismatched the glasses and tableware are and how the ambiance makes you feel like you are in Europe.However the food is bland and can easily be created by someone who can't cook. &#160;We have a Travelzoo coupon and were told we had to get a 4-course meal. &#160;We were confused b/c the coupon allowed for us to spend up to $80. &#160;We ordered a bottle (there were only 2 white options) and were told they didn't have it. &#160;They brought out 2 alternatives (an $8 bottle of Columbia Crest Chardonnary and an alternative I didn't recognize but chose). &#160;We were then served soup (best part of the meal but pretty much a can of cream of mushroom soup with fresh scallions and peas with some pepper).After that we were served salad: romaine lettuce, some goat cheese crumbles and some bottle dressing. &#160;It was the blandest salad I've never ordered.At this point we asked if we could order. &#160;Yes, we had not even ordered at this point and were busy drinking our crappy wine.I ordered chicken and couscous (hate ordering chicken when eating out but there were only 2 alternatives: pasta with meatballs and lamb chops, which my husband was ordering).Food was served promptly (not a good sign) and was not good. &#160;My chicken and couscous was lightly marinated and sauteed chicken strips over couscous mixed with a few peppers and (no joke) a cold, canned clementine orange. &#160;It was all different temperatures. &#160;I barely ate mine.My husband's one lamb chop was served with some red gravy and chickpeas over rice. &#160;It was very strange. &#160;Both of us were completely underwhelmed.They then served us ice cream (2nd best part of meal). &#160;It was just a scoop of vanilla with blackberry coule. &#160;It was good but easy to make.Service was strange but nice. &#160;They seemed like they wanted to turn over tables (fast food delivery, even if not ordered) yet were hard to find when you wanted to get out of there. &#160;They also kept asking if we liked the food, which should have been obvious with our full plates. &#160;This is one of those meals out when you wish you had just stayed home and made something better.If the food improves they could create a neat atmosphere. My concern is that they are all over Groupon, LivingSocial and Travelzoo. &#160;That should be a red flag as well.Hopefully they get better food soon! Two stars instead of one because the service was so good and they offered me something different to eat. That said, when you list a Potato Galette on the menu (Galette Pomme de Terre) you don't serve what resembled mashed potatoes mixed with chunks of meat. It also had dabs of a clashing tomato sauce on it and none of it tasted good. When I saw it I joked that it looked like something I'd get for breakfast at Glo's (in hindsight that was way too big a compliment.) The replacement dish was edible. Pros: Other than the the service the setting is lovely. The soup was also good. I purchased a Goupon to Thomas Street Bistro thinking we would enjoy a nice meal at what looked like a cozy, romantic restaurant. Yikes. The Groupon:\"for $48, you get a four-course French-Mediterranean dinner for two at Thomas Street Bistro in Capitol Hill (up to a $102.98 total value). The restaurant is closed on Tuesdays. Dinner includes:Two soups (a $15 value), Two salads (a $15 value), Two entrees from the menu or the entree of the day (up to a $43.98 value), Two desserts (a $15 value), Two glasses of house wine (a $14 value)\"Here are the low moments of the evening:1. Groupon said we needed to have a reservation. We called 4 times but only got the voicemail box of the owner. We decided to see if we could get a table anyway. 2. Tables are ungodly close...you have to suck in and pardon your self for sticking your ass in someone's dinner as you get in to your seat.3. Before even asking if we prefer red or white, my gf and I are both brought the house white. My gf says she actually wanted red. A few minutes later they bring her red.4. The soup we are brought is lukewarm and there isn't very much of it...but that's okay because it really is not appetizing.5. The \"salad\" is a plate of cheese and salami?6. My gf orders the &#160;Galatte de Pomme de Terre and I ask for the Salmon. I'm told they are out of salmon...at 6:00 on Friday night. So I get the Vegetarian Mediterranean Pasta. The beef my gf is served is RAW, beyond rare. It's inedible. My pasta is bland and nothing I couldn't have made myself. 7. The restaurant is chilly because the door is kept ajar. The waiters and owner keep opening the door every time my gf gets up to close it because she is cold. Finally the owner explains that the door must remain cracked because the fan in the kitchen doesn't ventilate properly. 8. Dessert is an apple-cream disaster that neither my gf or I finished.9. At the end of our meal a guitarist comes out, plants himself in the middle of the restaurant and begins to play. Nice...but unfortunately he is situated so he's playing right in my gf's ear. Really, the place is too small for that. 10. At the end of the meal the waiters and owner realize the beef is undercooked and offer to re-cook it. We decline and say we'll take it home (see pic). In sum, the food was bad and I won't EVER be returning to Thomas Street Bistro. However, the waiters were actually very nice but the owner seemed annoyed with customers. I think they may be hanging on by a thread because they are doing lots of Groupons and the quality of the food was really bad. &#160;I echo other posts in that it is totally food service food or bought from the local Safeway or QFC. &#160;If you are going to call your establishment a restaurant in Seattle, you have to be good, at least if you are going to charge restaurant prices and present yourself on your website as a place for good food.I have a good sense of humor so this comedy of errors made me laugh all night.I have never felt so rushed in my life, and their motto is: &#160;Sit Long, Talk Much, Laugh Often. &#160;We did laugh, but AT them. &#160;We had a groupon, and once seated, the waiter (very inexperienced) rushed over and apologized for the wait (seriously, we had just sat down - first giggle of the evening). &#160;The first thing he said was \"Do you have a coupon or something?\" &#160;Were we pegged or what? &#160;Then he said, \"okay this is how it goes: &#160;you get a soup, a salad and your choice of main dish.\" &#160;Fair enough, although that is not what the groupon said. &#160;He complained about being tired because it was so busy \"from all the coupons\". &#160;I said, well, busy is good for business, right? &#160;He said, yeah, well financially I guess. &#160;I know 12 hour days are long on your feet, poor kid, but... &#160;Later, when ordering, I asked what was in the soup, he said it was just vegetables \"such as white broccoli, peppers, etc\" &#160;\"Such as\" or really? &#160;hahaThen we picked out a bottle of wine. &#160;The waiter told us that they didn't have it, that in fact, none of the wine on the menu was available. &#160;He said he'd ask his boss for a suggestion. &#160;He came back to the table with a bottle and started opening it. &#160;We asked what it was, and he had to read the bottle and read the label to us. &#160;It was some cheesy private label wine. &#160;My BF asked what the other choices were - evidently there were none. &#160;Then we asked the price. &#160;\"Oh $36, they're all $36.\" &#160;Ooo-kaaay.The owner came to the table and asked us some questions but did not listen to a word of our replies- we could tell by his reaction that he cared very little. &#160;The food, oh god. &#160;I admit, I'm am spoiled and am so used to fabulous food in Seattle. &#160;I'm also lucky to have an amazing boyfriend who cooks like a dream. &#160;But seriously, I think I could have made our dinner when I was in junior high school. &#160;The salad was literally 20 pieces (no exaggeration) of wilted mixed baby greens, one unripe cherry tomato and, my boyfriend's quote, \"some Paul Newman's Italian dressing.\" &#160;The soup was a puree of vegetables, overly salted (for extra flavor?). &#160;The waiter brought my entree within 3 minute of my salad on a HUGE plate (remember, this place it itty bitty). &#160;I just looked at him, then down at my 20 leaves of lettuce, then back at him... like where am I going to put that? &#160;The waiter said, \"can't you just push your other stuff over?\" &#160;My boyfriend said \"we're not in a hurry or anything\".Anyway, it goes on and on.At the end of the meal, the owner came over to check on things and started to take our wine bottle away, which still had 3/4 of a glass left in it! &#160;My boyfriend said \"whoa, let's leave that here, shall we?\" &#160;A totally bizarre experience that I cannot think was a series of 20 or more odd occurrences. &#160;This place had opportunity and may have at one time been very good, but now, it is crossed off the list.The guitar player was nice and sweet. My husband made reservations a week in advance and we drove from Issaquah to try this place. &#160;We got there and saw a huge crowd of people outside. &#160;The owners triple booked the reservations (at least) the place has like 6 tables and there were 20 people outside in 40 degree weather. &#160;Then, the owner or manager or whoever took the couple who had later reservations than us because they had arrived 30 minutes early for their reservation. &#160;So I guess what's the point of making one if it doesn't count? &#160;After waiting in the cold for over 20 minutes we left. &#160;We will NOT be back. &#160;A customer exiting before that said the food was , \"ok\", I guess it wasn't worth the wait anyway. &#160;We went to Daniel's Broiler instead and had amazing food and amazing service. &#160;Sorry for trying to support small local businesses! &#160;I still do that, but this one is not worthy. The dining room is about 12'x12', so the tables, and therefore people, are gently crammed in. &#160;The acoustic guitarist sat RIGHT behind my husband, knee touching his chair - nice music but difficult to hear the conversation.My mushroom ravioli was good, but the other three people's food was mediocre at best.The owner walked around and put a large glass on each table, telling us it was for tips for the musician. Tacky!We all agreed we wouldn't return to Thomas St Bistro. Truly one of the worst dining experiences I've ever had. Do yourself a favor and don't ever go there.Offensively TERRIBLE serviceMediocre foodOverpriced for what you getI'd give a full account of the night, but after reading some of the other 1 star reviews it seems like it would be redundant. Even if you think you can handle the bad service, here's what you might be in for food wise:-Completely inaccurate wine list.- Starter salads &#160;- limp iceberg lettuce with bottled dressing and one tomato- Chicken Ravioli (dinner special for $21.99) &#160;a) had no chicken in it &#160;b) seemed like it came from Costco &#160;c) overly seasoned with dried oregano &#160;d) served on top of iceberg lettuce - WTF?!?!-Pork Chop a) REALLY tough b) rice that came with it - incredibly dry and flavorless - seemed like a packaged type rice like Rice-a-roni or something.-Dessert - terrible mini cream puffs (also very Costo-esque) With a weird \"no reservation but I'll put your name on the list\", VERY skimpy portions (was it because most of the patrons were coupon holders and so we get less? The too-cold &#160;tasteless sliver of cheesecake was about one inch at the wide end), the bland, flavorless soup (needed salt badly, but no shakers), \"no salad in winter \" policy instead just a tiny piece of toast with a dab of salmon mouse, NO parking, AND a \"tax bill\" on my coupon for $10.50 which shows they valued this poor dining experience at arount $100, so, NO! I will never venture to this place again. It was definitely NOT worth it. &#160;A few good points: My small glass of wine was very good, &#160;and the smallish portion of rigatoni was tasty (this entree was priced at $20!) &#160;The owner came over and apologized for \"being stressed\" &#160;and &#160;for \"the service\" &#160;which was quite good, by the way. &#160; &#160;Instead he should have apologized &#160;for &#160;the ridiculous \"tax\" bill, &#160;the skimpiness of the portions, the bland soup, and the weird seating game.And I had to call about seven times in two days to get on \"their list\". This place is AWFUL. I went last weekend and will never go back. The service was horrible and the food was mediocre and overpriced (and not as described on the menu). We arrived about 20 minutes early for our 8 pm reservation and were immediately escorted to a table, without being asked if we had a reservation or asked for our names. By the time 8 pm rolled around, the place was packed (not hard to do with only about 10 tables jammed in a space about the size of a small bedroom), and it was apparent that if we had just arrived on time for our reservation that they would not have had a table for us and we would have been waiting outside in the cold for a good long time. Whatever. We had a table. We weren't outside. It's all good, right? Not so much.Included with the $20 entree was supposed to be a bowl of creamy vegetable soup and a lentil &amp; feta salad. As soon as we sat down, a waiter brought us each a saucer with a small slice of cheese, a slice of prosciutto and a point of bread. Apparently that was supposed to be the evening's second course (in lieu of the salad, with no explanation regarding the substitution), but since it was delivered to us as soon as we sat down, &#160;we had to ask about the soup in order to get it brought out to us. So we asked. More than once. The soup arrived. And it was ... eh. We had heard that the soup was not to be missed (which is why we were so determined to try it), but it was just OK. Creamy, sure, but also one-dimensional and underseasoned.My husband was going to order the pork chops until he saw a neighboring diner send hers back. I was thinking about the chicken in lemon cream sauce until the table next to us ordered it and I saw that the sauce was the unappealing yellowish-brown color and texture of baby poo. I tried to order the smoked salmon salad, but they were out of salmon. I opted for the vegetarian pasta (linguini with peppers, tomato, olives and a harsh-tasting feta), thinking that would be hard to screw up. Hubby got the rigatoni bolognese. Both were fine, nothing special. And yes, my plate of proteinless noodles cost the same $20 as my husband's meaty pasta tubes (and the pork chops and the chicken and etc. etc.).When we ordered our meal I ordered a $55 bottle of red wine that was the most expensive on the small list. Instead the owner brought me a bottle of cheap French wine that was not the one I ordered. He said it was one of his favorites and that he would not be charging us $55 for it since we had been waiting a while for service. After tasting it (at his request), I said I didn't like it since it was about 6 years past its prime (it was a 2003 Chateau Pech Rosie). He brought out a different bottle (nothing from the small wine list, mind you), which I also said no to without opening/tasting. So the first, unacceptable bottle sat on our table, unpoured and undrank, and the owner tried to charge us $38 for it when we asked for our check, then briefly argued with us about it until I firmly stated that a) I didn't order that bottle of wine, b) it was not drinkable and c) there was no way I was paying for it. I think the only reason he didn't argue further was because the place is so tiny that he didn't want everyone else in the dining room to hear my complaints, since there were other diners who seemed just as disgruntled as our table.If both my husband and I hadn't been awakened in the middle of the night with gastrointestinal distress, the evening would have been comical, but as it is it was just irritating. We went for dinner with a Groupon which seemed to turn off the owner. He explained that we could either have the \"special of the day\" or order off the menu. &#160;We ordered two specials (veal) and one pork. &#160;The salad came, a bit of iceberg lettuce, cheese and one slice of tired tomato. &#160;That lead to the mains, veal with mushroom sauce and pork with tomato sauce. &#160;Both served with a bit of rice and a lot of attitude. &#160;Then the desserts with the specials, creme puffs from Costco. &#160;All in all, McD's has better service probably a better choice. Summary: &#160;The worst excuse for \"fine dining\" I have EVER experienced (and I do know what good fine dining is supposed to be). &#160;Kept expecting Gordon Ramsay to emerge and say we were part of the \"before\" crowd on Kitchen Nightmares. &#160;Just embarrassingly bad.Details: &#160;There was not a single thing good about this experience (neither was there anything remotely \"French-Mediterranean\" about it). &#160;I kept expecting (hoping?) someone to come out and say \"just kidding, you're on a hidden camera show and the restaurant isn't really this awful\". &#160;The food was laughably sub-par and the \"service\" was lousy. &#160;Was supposed to be a $103 value (over $50 per person), but I could have gotten better quality ingredients from the grocery store (and prepared them more convincingly) for less than $20 total. &#160;Dishes and flatware looked like they had been purchased at a garage sale, weren't convincingly clean, and they didn't even have enough clean sharp knives to give us more than one for the table. &#160;I couldn't wait to get out of there, and it seemed the wait staff and the owner/manager (who never once came to our table to ask how things were, even though there were only four or five other diners there and he kept lurking around with apparently nothing better to do - and our server clearly knew we were dissatisfied) were of the same mind.4-course meal: course 1 (their \"award-winning\" soup) was served cold (tepid at best), and tasted like it was poured out of a can and sprinkled with freeze-dried herbs.Course 2 (the appetizer) was not at all what the menu promised - instead, we got a cold plate of thinly-sliced salami, a slice of cheese, a slice of tiny bread, and two small green olives. &#160;The free salami and cheese I had at a winery earlier in the day was far superior to this joke.Course 3 (main course) - two of the menu selections that looked appealing (salmon and chicken) were not available at all, and the item I had my heart set on (the lamb) was being \"replaced\" by a flank steak. &#160;So I couldn't order any of the things I actually wanted - and no special was offered (even though the menu mentioned a daily special) - so I ordered the pork chops. &#160;We were told it was a \"house favorite\" and that it would be cooked medium-rare. &#160;The menu said it would be served with rice and a wine tomato sauce. &#160;What I got was a very thin chop that was overcooked and leathery, not medium-rare, with an overwhelming sauce that tasted more like salsa, served over bland mashed potatoes instead of rice (we were not told about the switch). &#160;Oh, and there were two small, limp, lifeless slices of zucchini on the plate. &#160;Just embarrassingly bad all around - I can't cook, and yet I can manage to make boneless pork chops at home that are tender, juicy, and beautifully seasoned. &#160;This bone-in chop was too thin, overcooked, and not seasoned, unless you count the salsa that seemed completely out of place on the dish. How the mashed potatoes managed to be so devoid of flavor suggests effort on the part of the kitchen staff to remove it (or at least avoid adding any).Course 4 (dessert) - I predicted before we got it that it was going to be a slice of defrosted frozen cheesecake from Costco, but somehow I still managed to be disappointed when I discovered I was right. &#160;It was tiny, still partially frozen, and the chocolate flavoring tasted like Jello pudding. &#160;Not exactly fine dining, which I was under the impression that this would be.The house wine tasted like it was probably Charles Shaw from Trader Joe's or something along those lines - granted, we might have gotten better off the wine list (assuming they had one), but even for a house wine this was just nasty.Again, this was NOT worth anywhere near the $103 \"value\", nor was it worth the $48 Groupon price. &#160;They should be ashamed to charge more than $10 per person. &#160;My $15 bowl of risotto I had at a different restaurant for lunch was, on its own, worth more than all four courses and the free wine from Thomas Street Bistro.I am disappointed, frustrated, and frankly quite angry about the waste of money and time this experience represented. &#160;I wish I had read some of the other reviews on Yelp before I bought the Groupon, since this was clearly not a one-time off night for them - it is obviously a pattern of bad service and sub-standard food.I had an exceptionally bad experience, and normally I am not very picky. &#160;But this was just unreasonably bad. &#160;I will NEVER go to this place again, and I will encourage anyone who asks me to avoid it at all costs. We had purchase a voucher from Travelzoo for $47 to get two glasses of wine, soup, salad, entry and a desert. On Sunday night they did not have soup, salad and entry was inedible and we got a food poisoning just trying the stuff. They offered us \"free desert\" and brought 1 1/2\" piece of cake from Safaway. At the end they brought us additional charges for $17. Unbelivable bad service and terrible food. I don't know where to start. &#160;Absolutely horrendous and appalling comes to mind. &#160;When I ordered my wine I was told sorry we don't have any more wine glasses, you have to wait. &#160;After 15-20 mins I received my glass of wine, warm (it was a white), in a greasy, dirty water glass. &#160; When we finally got someone to come over to take our order, we were asked if we were there using a special/coupon. Reply was yes and it went further downhill from there. &#160;When the food came out, his spaghetti in a child's portion was cold and congealed -- AND he had to wait 10+mins for a fork. &#160;My veal and pasta dish came out 12 mins later, cold. &#160;The veal was tough and rubbery. &#160;The side of pasta was cold, sticky and dry -- no sauce, not even salt. &#160;I don't like big portions and am always happy to get smaller portions but this was ridiculous -- his portion was a child's portion -- I made him a sammy when we got home. &#160;The dessert was a pre-sliced cheesecake obviously from Costco or some other warehouse. &#160; &#160;BTW, if you're a couponer you get a smaller, crappier portion as we happened to notice the comparison with other 'couponers' vs paying out right. &#160;Whatever you do, don't go. &#160;You'll be absolutely horrified. &#160;Where's Gordon Ramsay when you need him -- but I don't even think Gordon could save this place. It took me a while to decide how many starts to give Thomas St. I think the reason for the inconsistencies in reviews (mostly just 5-star or 1-2 stars) is that the service and quality is inconsistent. I've been here about a dozen times over the course of two years, sometimes with coupons and sometimes without. I don't know if I've ever seen the same waiter twice, but often I'll see the nice owner milling around. I recently went for weekend brunch, and the food was pretty tasty, but the waitstaff seemed like they were in over their heads.When I first started going here, we'd often get a complimentary glass of wine and an appetizer in addition to the soup and dessert. Now I think it's just the soup and dessert. The food is usually something I could have whipped up at home, but I do like the cozy atmosphere. Sometimes the food is pretty good and sometimes it's bland. I wish it was more consistent.I want to give this place more stars, because I do like the cozy atmosphere and the nice owner, but I'm probably being generous with 3 stars. this place needs chef Robert Irvine...I am going to submit a food network requests. &#160;it appears Yelpers give it a mediocre (ok) but i think even that is an overstatement.... it's literally Seattle's smallest restaurant space aka uncomfortable...The dishes or what they call four courses were forgettable as I wanted to forget about being here. &#160;They were poorly constructed and most things looked like a hodge podge one pot cooked at the same time.. There was very minimal protein where it shouldn't warrant the ingredient of it in the dish. They didnt have a beer menu and pulled whatever was left in their fridge, lucky me got a Guinness. Definitely a Groupon and http://restaurant.com trap as others may think not.. The location is cute, but the food is not good. It was very bland and not filling. I also did not like that the staff was always so close to my table. My husband and I felt like we couldn't talk because we had someone listening to our conversation. We bought the Groupon and vow never to go back again. Save your money."),
      "categories":"Greek,Mediterranean,French,Restaurants",
      "zip_code":"98102",
      "review_count":42,
      "avg_rating":1.7916666666666667},
    {
      "reviews": preprocess_text(" It's about time! &#160; A bonafide, no BS, pizza joint in Seattle! &#160;By far, the best &amp; most legit pie I've had in Seattle since moving out here from NY over 22 years ago... &#160; Crust, sauce &amp; toppings- all terrific. &#160;The people running the place are as genuinely friendly (also an east coast trait!) &amp; professional as can be. &#160;Price wise- a great value. The overall vibe of the place reminded me of home, actually made me a little homesick when I walked in. 1 warning: Not being \"in the know\", I got the large for 2 of us. &#160;Ridiculously huge, by any geographical standard! We'll be eating great pie for a while, which I'm more than ok with, storage is an issue though. &#160;If you're feeding 4+ ppl, or you have a big fridge, great... &#160;Otherwise, sounds like the small works for 1-2 ppl. &#160;Oh yeah... Good Greek Salad too!Thank you Italian Family Pizza for bringing real pie to the people of Seattle. &#160;Don't change a thing and keep up the great work! So, okay, here it is..THE place for East Coast style pizza in Seattle. Being from MA and also 50% Italian (mom's side) I've been missing shops like this quite a bit. Italian Family Pizza might sound like a sort of odd name, no? Maybe, but it is accurate. Let's break the name down...in reverse order...I don't know why.Pizza:Comes in two sizes..small and GINORMOUS. No slices are served. I can finish a small on my own no problem but have definitely seen people share one. The large is huge. Seriously, I woke up one morning after a night out and saw this huge box taking up my entire kitchen table and thought \"well f me...drunk farz bought a tv last night!\" Nope...it was a pizza from Italian Family Pizza.Style wise the pizza has a thin crust and a slightly charred bottom. This crispy crust holds the delicious sauce and cheese just right. You can fold and eat and the slice will not split. It's so good. For toppings I've stuck with pep but have heard rave reviews for the clam topped pies so that might be my next order.Family:True story I was in eating by myself after Christmas shopping and a guy and his daughter walked in. He asked to buy a sprite and where the bathroom was for his daughter. As he's being cashed out..Steve (owner?) says \"Did you really want this Sprite? or did your daughter just need to use the restroom?\"The guy: \"oh, no. She just needed to use the bathroom\"Steve:\"Then you don't need to buy the Sprite, you don't have to feel obligated to buy anything\"Who does that now-a-days? Cutting onions, man, I tell you.Italian - So Italian it hurts. Dominque and Steve..hysterical and so nice. Whenever I've been there Dom has always been checking up on tables, striking up conversations and making the customers laugh. The banter between the folks here remind me of every dinner I've had back home.Anyway, that's enough. Stop in for the pizza and the atmosphere.ps they have great beer. Downtown pizza lovers unite! &#160;Unite at this new pizza joint! &#160; It's a prime location for the Pike Place Market and Westlake workers. &#160;I, however, traveled from Pioneer Square. &#160;I wish I had more time to chill out. &#160;Because of my restricted hour lunch I had to box it up and take it back to work. &#160;The restaurant had a lot of tables. &#160;It makes sense...the restaurant name IS Italian Family Pizza. &#160;I could also tell it's a family oriented spot because of the many photos on the wall and the cluster of customers who walked in. &#160;The service was very warm and welcoming. &#160;I had the meatball pizza special. &#160;The meatballs are house (or home) made by the wife (owner). &#160;I waited for about 10-15 minutes. &#160;The pizza was made by the husband (also the owner) was delicious. &#160;I love this unique pizza place. &#160;Downtown has nothing else like it! This is real Brooklyn-style pizza, not the arugula-ized crap that can be found all over Seattle. The dough is crisp and a little burnt on the bottom. For the base of the pizza, you can choose among tomato, cheese, or white. We selected cheese, but still found their tangy homemade tomato sauce to be the star. The topping selection is pretty minimal (you can count them using both hands), but they have the basics covered. We got ours with pepperoni, mushrooms, and roasted red peppers, which were generously spread across our pie. The pizza is a bit pricey with a large cheese costing $22 and a topping is $3 each; however, we had so much left over that we had to tote the slices back in two boxes. It was like a walk of shame out of the place. One last thing - friendliest/funniest family ever. So a guy walks into an elevator with a hee-uge pizza box... This could be the start of a joke, but the joke's on you if you don't give this place a try. This is the place to escape from braised kale, gluten free crusts made out of nuts, and vegan meatless \"meatbalz.\" The pizza is a true taste of the East Coast. It comes in two sizes: 10\" and 23\" (although if you ask Dom, he'll insist the crust adds an important extra inch -- boys!). The giant pizza is the reason for the giant box. The tomato sauce is the brightest and freshest I've tasted in town. The crust is just right. The house salad is well worth the room it diverts from pizza eating. But when you add Dom and the crew, it's like you've been invited to eat at the cool kid's house down the block. Stop reading reviews. Go. There. Now! I was going to write this review a while ago, but wanted to do it well and not rush it.With that said:OH MY GOSH! FINALLY!!!All I remember about the fateful day I discovered this place is that I had trouble fitting the pizza box in my car, and that all I could say while eating it was \"This is so @#$#ing good\" over and over. &#160;I think I might have even cried.My fianc&#233; and I are east cost natives who recently moved here for work. &#160;We searched around a while looking for a little piece of home to no avail, and then our prayers were heard. &#160;I am not saying this will be your favorite pizza. &#160;I am saying that it is the best east coast pizza I have found out here, and I will be going back. &#160;For the record though, it is some of the best pizza I have ever had (topped only by that one place in New York I found when I missed my bus...). Anyway, give it a try, you wont be disappointed. &#160;Plus, owners and staff are very friendly! So far I've tried the cheese pizza with sausage, mushrooms, and pepperoni, the stromboli with pepperoni, mozzarella, and tomato sauce, the white pizza, and the meatball sandwich. &#160;All have brought back memories of meals eaten in Philadelphia. &#160;If I didn't know I was in Seattle, I would have thought I was in the City of Brotherly Love!The sauce is made from scratch as are the meatballs. &#160;I'm really picky about sauce, and Jennifer's sauce is just perfect. &#160;The meatballs are made with a blend of meats from Don &amp; Joe's at the Pike Place Market, and I'm really glad they have them every Monday. &#160;The meatball sandwiches are super-popular, and if you want to make sure you don't miss out, you can reserve one!Steve is super-friendly and welcoming, and he insists on making everything fresh. &#160;No slices here, which may deter some folks, but don't let that deter you! &#160;It only takes slightly longer to get an exponentially better custom-made pizza.Recently I organized a large group event, and Dominick had everything reserved and set up for us well before the start time. &#160;He took time with everyone to explain the variety of offerings and made sure everyone had everything they needed.It's a real family affair &amp; I wish them much success!! Checked it out for lunch recently and it's gonna be a new option for the downtown working crowd. &#160;No slices are sold here but they do have individual pies if you are not in a larger group. Ordering goes like this.... Pick a type of pie (white sauce, cheese, or red sauce) and then pick your toppings. &#160;$1 for each topping on a small and I think $3 per topping for a large. &#160;My 2 topping small pie with a soda came out to around $10.The red sauce pizza, ooops...I mean pie, doesn't come with mozzarella and is more sauce then cheese so I would stick with the cheese pizza base and go from there. &#160;Overall pizza thoughts - Good crust, little heavy on the sauce, could go with a few more toppings but great for lunch. &#160;I would definitely stop by for a pizza again and it is filling the void of a lunchtime pizza option not well represented in the area. TRENTON MAKES THE WORLD TAKES !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!&#173;!That's all you really need to know about this place. They make the best pizza in Seattle and it's all yours for the taking.Jen and Steve CALOZZI &#160;put there heart and soul into bringing real pizza to Seattle. The joint is family friendly and nothing but laughs and open arms. The food is amazing my favorite is the Tomato pie with garlic and sausage. Thats right no cheese maybe you Seattle folks &#160;will get what a real pie is one day but till then feel free to &#160;order the cheese pie as well. They come in 10\" and 24\" ( I eat a 24\" to the head the first time just cause it reminded me of home)Salads and wine/beer go right along with the east coast flavor. So give it a try and enjoy all the love as you &#160;walk right in to the joint. This place serves delicious, very saucy pizzas for a good price. $7 for a sizable personal pizza, tax included! It feels like a great deal, and I definitely leave feeling full. And I don't think they're lying about the Italian Family part-- they all look related. :-)Downsides are the wait times and the relatively narrow list of toppings available. This shouldn't deter you though, unless you absolutely need chicken or kale on yours. Not much to say beyond 'the best' - excellent food, great people, amazing service. I'm a New Jersey native living in Seattle, Washington and always miss NY/NJ style thin crust pizza. I've finally had the very best NY/NJ style pizza here in Seattle. Thank you Italian Family Pizza for providing us with excellent east coast flavor. The best Pizza in downtown Seattle for sure, and although I won't pretend I have a comprehensive experience of Pizza places in the Northwest, this is some of the finest pizza I have ever had the pleasure of eating.Come with a few friends and get the 23\" pizza. The sauce/toppings/crust ratio is awesome. Sweet, rich, tangy tomato sauce, creamy cheese, crispy bottom AND chewy, delicious dough above, and toppings bursting with flavor. short and simple. best pizza in seattle. love at 1st bite. Stopped by on a friday night to try it out. &#160;Easy enough to order small cheese pie to go. &#160;Loaded with sauce and cheese and tasty. &#160;BUT, &#160;got a pie that the bottom of the slices were covered with excess &#160;flour powder &#160;or maybe some cornstarch. &#160;Perhaps it was a mistake and pie was dusted by accident but it just didn't make it very appetizing. &#160;probably just a oversight but I'd probably go with order from primo instead in future.I went back a week later to try it again--owner saw review and very sorry. &#160;the next pizza was very good! &#160;And actually better than primo. &#160;So forget what I said before--just bad luck--It sounds like they wouldn't let that happen again. We ended up here after finding Post Alley Pizza closed and man, are we glad we did! &#160;Closest thing to east coast pizza that I've had anywhere on the west coast. &#160;The staff were very friendly and have a good dose of Italian-American east coast roots to boot. &#160;We had a large tomato pie and were in love with the first bite. &#160;Topping list &#160;was pretty standard (nothin too creative here aside from clams). &#160;Highly recommend. The \"pie\" is the way pizza should taste. This is what I grew up eating in the burg in Trenton. Crust is crisp and has bubbles! It's a Jersey thing. And you can fold the slice. The photos on the walls are a pictorial of Philly, Trenton, and NY. Hands down, the best \"pie\" in Seattle. No slices, but the service and short cook times insure that your \"pie\" is fresh. The way it should be. Best pizza in Seattle, and you can't beat the service. Jen and Steve truly care about each customer as much as they care about each pizza that comes out of their oven. Take it from an east-coaster: this is the spot you want to eat at.My favorite: a tomato pie with clams. Delicious! This is the best pizza in downtown Seattle. &#160;Clearly, a lot of love has gone into perfecting this Trenton style pizza. &#160;The folks who run this pie palace are about the nicest folks you can imagine. &#160;I've searched and searched for good pizza in downtown Seattle, and am glad to have finally found it. The quality of the ingredients and the size of the pizzas are fantastic. Best pizza in the Northwest! I love the sauce and pizza crust! The restaurant is really family friendly too. So glad to have such a great week night restaurant downtown! Great Pizza! Truly the top of the non-napoli style pizzas in Seattle. &#160;No wood fired fancy nonsense, just good old NJ/NY style pizzas and Trenton-style Tomato Pies! Fast service for personal and full size pies - get down there now, the line will be out the door as soon as the papers/blogs blow this place up."),
      "categories":"Pizza,Restaurants",
      "zip_code":"98101",
      "review_count":22,
      "avg_rating":4.6956521739130439},
    {
      "reviews":preprocess_text(" I ran into this place by accident. I never heard of it. I never ate it before. From the long lines and countless articles on the exterior window, this place looks like a must. I had the chicken mushroom and rice piroshky. It was highly suggested from the staff. Even Chef Anthony Bourdain came here!I want one right NOW! Flaky. &#160;Flavorful. &#160;Fuckin' delicious!If you are visiting the Pike Place Market, you have to stop by Piroshky Piroshky first for an on-the-go snack that you can eat while walking through the market . &#160;Got a couple of minutes? &#160;Stay and watch the workers make them. Ok, I will be up front with you. I saw this place on Anthony Bourdains \"No Reservations\". I am a sucker for anything Mr. Bourdain raves about. I knew when I got to Seattle that I had to make a visit.So...my first day in the actual city. My husband goes to work...guess where I go? This place. I guess I am lucky because I eat at weird times of the day...this ='s no lines! I walked right in and immediately was the only person in there. It is a very small place...with only 2 stools to sit at. Basically...you go in...order...and then be prepared to find somewhere else to eat.Since it was my first time here I went with the standard...the beef and cheese Piroshky. I am sure most of you already know that is was amazing. The crust was perfectly cooked and the meet in side was delicious. I happily walked down the street stuffing my face with it. It was just a few short hours ago that I consumed that delicious meet pie....and I am already thinking about another. In the morning, we drove nonstop from Eugene, OR to Seattle and were famished. Pike's Market was a stop on our road trip, but we only had an hour to spend there. Based on a friend's recommendation, I made it my mission to get a piroshky here. Not knowing where anything was, we walked around and asked several vendors before finally finding this place.OMG. It was so worth it. The ham, spinach, and cheese piroshky was delicious! Light and flaky on the outside, warm and hearty on the inside. Sitting there in cold and damp Seattle weather, I could see the steam coming out of the filling, and for some reason, that made the piroshky taste even better.Unfortunately, we had to go to a friend's house for dinner. It took everything I had to stop myself from trying a sweet one and ruining my appetite! I will definitely be back. There really is not better place for meat filled pastries. I'm down with a humbow or 2 but I could easily ingest 3 to 4 Piroshkies. Not that I need to and it's probably not really recommended but they really are that good. Think about it... hot fresh soft dough, braided and baked around seasoned &#160;meats, potatoes, cheese, mushrooms, broccoli, spinach... not to mention the sweet options. In fact, I still have a bit of Holiday shopping to do... I wonder how long a Piroshky will keep... I really don't know what the dealio is with this place. It has tons of stars, and we were suckered in to stop to watch a manly man fold pastry. Not to mention like the Boudin in SF, they use this big fan to blow the scent of yummy baked goodies onto the sidewalk to lure you in. Well you know what? Like that lame Yellow Pages ad on the radio, it worked. I walked in. And shelled out 3.50 for a piroshky. I looked over the selections (and this is post HUGE breakfast from Portage Bay), and picked out a smoked salmon one. The crust looked super flaky... but I was mildly disappointed by the texture... It was no different from the regular buns I can pick up at my neighborhood Cantonese bakery. It was just... a filled bread thingy. :(I have to say though... the filling was VERY yummy... and looks can be deceiving... Being completely ignorant, I thought the crust would be similar to a croissant... flaky and buttery. Oh well!! Should have checked out the yelp reviews next time. :) 4 stars for the nice show, the yummy filling, and the tourist appeal. This place makes me horny,As in I get hot as hell for a Piroshky!All around yum yum in my tum tum!!!Just take the advice of all the great ratings and get there, as I think it's obvious that there is no possible way you would be disappointed...unless you are not a fan of goodness? I've had real, authentic Russian food, so when it comes down to it, I'm very picky about the Russian food I do eat.. And I've got to say the only Russian bakery that I love more is a hole in the wall in San Francisco. Everything is delicious, the staff are always insanely busy but so nice and helpful if you have questions.Oh Piroshky, Piroshky, some of my favorite Seattle memories have you in them. I've even bonded with a woman from Arizona, while I was living in California, over you. When I was living downtown, near the market, I would go there every Sunday morning and get a piroshky and sit on the dirty sidewalk in a cute skirt and eat my little present to myself.Go here. You won't be disappointed. I walk by this place everyday on my way to work. Nothing hits the spot (especially if i'm hungover) like meat and cheese in a pastry, especialy at 8.15 in the morning when there's no line. I'm in and out in seconds.  This place is quite possibly my new obsession. I have a plan for 2008 to eat one piroshky per week until I've tried them all. Anymore than once a week and I'll start looking like the nice Russian lady who bakes them.However, the best thing about this place is the tip cup it says \"The Cold War is over, tips are appreciated\". Hee! I've got such a hard-on for Piroshky Piroshky, that sometimes it's tough for me NOT to flip the bird at work and head down there for lunch . . . . at 10a.m. &#160;. . . . . The breads are freshly baked and the fillings are delightful. &#160;I'm going to try my derndest to eat one of EVERYTHING there before the day I die. &#160; Although I wouldn't mind if they seasoned their fillings a smidge more (especially the ones with p'tater fillings) I'm prolly asking too much . . . . Because snagging a Salmon Pate Piroshky in under 2 mins for only $3.50? &#160;Brilliant! &#160;Now I can absolutely eat my Piroshky and have it too - there's a case full of 'um! This place reminds me of my days spent in Moscow, thats how authentic this stuff is...ok I've never actually been to Moscow but seemed a good way to start my review.....Whatever the case may be (authentic or not authentic) it's all FREAK'n delicious. &#160;The beef and onion my substitute when they did not have the beef in cheese literally melted in my mouth (seriously that's a good thing). &#160;The pastry is filled with buttery goodness.... and look theres meat inside...I'll take another please. Why, oh why, did I try this place? I knew it was going to be good; I just didn't know how good. I am now addicted and I can't stop the cravings! Why can't I live closer to the market? I love the beef and cheese and the smoked salmon shaped like a cute little fish ready to be caught. Everything on their menu sounds delicious, but I always lean towards the savory.Maybe I should buy a bunch the next time and freeze them. Hmmm...I wonder how well they freeze? Ideas:Pick one:A. You are at Kells and just can't get behind any of the menu items.B. You are going to Kells and know you just don't want a corned beef sandwich.Then, is Piroshky Piroshky still open?If so, proceed, if not, find you're out of luck.Now, walk down the hill and to the left, hop in longish line your come across, that's Piroshky Piroshky. &#160;You wont know what you want until you have to order, simply by nature of how the counter is set up, so get ready to think fast.3 Sweet, 3 Savory, pick um.Now smuggle them into Kells, they won't care. I have a soft spot for salty breakfast pastries, hence my love for the kolache (Czech pastry). &#160;So I was so excited to go here!!! As recommended, I got the smoked salmon piroshky (if you're in Seattle, you gotta get everything seafood) and the beef/cheese piroshky. &#160;The smoked salmon was so delightful! &#160;I was actually expecting slices of smoked salmon but it's more of a pate. &#160;AND, what's even better is that I normally really dislike dill but the dill in this was so light that it complimented the salmon very well. &#160;The beef/cheese piroshky was not bad... lots of onions in it and not as much cheese as I thought.They are a bit pricey at around $4-4.50 per piroshky but they are big... you should share it with another person if you'll be eating other things at Pike's Place. &#160;My only recommendation is that they make them half the size for half the price! &#160;That way I can have different kinds without being overly stuffed.I'm back in California &amp; I miss this place already. This place is amazing. &#160;I literally have nothing bad to say about this place; the piroshky is always fresh and the staff is always friendly. &#160;I haven't tried all of the many different flavors of piroshky, but the ones I have had (which include the beef and cheese, rhubarb, cinnamon and cardomon braid, cranberry apple, and plain beef) are wonderful!They don't always have borscht, but when they do it is also great and I highly recommend it. Everyone is giving this place tons of stars for a reason IT\"S GOOD STUFF ! man , it's so good it'll make you slap yo mommameat cheese, and the white choco cherry mmmmmmmmm ..... I am giving a five star review despite the fact I've only had one thing here - the cheddar and green onion piroshky. &#160;I had it twice, in back to back days, each time on a full stomach. &#160;I had to force this thing down since I had no more room for food, but it was absolutely worth it.This magnificent bun is served warm, with the green onion providing a fresh bite that counterbalances the creamy cheddar. &#160;The size (think softball pounded flat) is perfect for breakfast or, in my case, a snack on a full stomach.I also tried a bite of the cherry and white chocolate piroshky, which left me similarly floored.Go here and get FoodHappy. &#160;It's the best emotion ever. These piroshkis are good just about any time of the year, any time of day. The ones that come in the flakier crusts (like the sausage or the ham and spinach) tends to be a bit on the greasy side. I prefer the more \"bready\" type piroshkis. &#160;I'm also not a huge fan of the sweeter stuff but that's just me. For those of you who are visiting or new to town, here's a great way to spend your summer day: walk around the market and grab some fruit, get a piroshki, go down a few doors to the Mexican grocery store and get a mexican coke (made with real sugar, not corn syrup) and find a spot of grass. It will be about the best $10 you can spend in the city. Yum! &#160;I always drag my out of town guests to this place and make them try the Salmon Pate piroshky (my fave). &#160;Most of them fell in love with it and always want to go back for more whenever they visit. &#160;Its awesome!The only downfall of this place though, is that its located in a box. &#160;I don't think more than 4 people can fit in there at a time. Piroshky, Piroshky had me wanting more, more.There are a lot of food options at Pike Place Market. This is the place we kept coming back to. Piping hot piroshky made a good hand warmer cupped between my palms. Unfortunately for my hands the little meat pocket was better in my belly.I was partial to the piroshky with beef and onion. Roller coaster of emotions eating one of these. The smell and taste of one will make you giddy with anticipation. &#160;Watching it slowly disappear into your mouth will make you sad. Until you get another one! And then the vicious cycle begins again. I think that's why I had six of these. Maybe they should make them bigger. this place is by far my favorite PNW food passion. I enjoy them weekly, well as much as i can get away to Downtown Seattle to start my weekend. You might have to wait in line for a few minutes, you will have to rub shoulders with tourists, you will go broke if you visit frequently :) My favorite piroshky is the cheddar and garlic...yummmmmmmy :)get two, one for instant gratification and one for later in the afternoon when you are craving the yumminess. What more can be said for this place to let you know that it is a Seattle must visit?love this place more than my own cooking, well you know what i mean. My mouth is in love with this place. &#160;Really, just about all my senses were in for a treat while waiting in line for my fix. Numerous peds walking by commented that if so many people were willing to wait in the long, and quickly moving line, it must be good. And the counter staff does a good job of keeping things moving efficiently. This time around, I had a bite of my mom's apple cinnamon roll (delicious!) and the sauerkraut, carrot &amp; onion (orgasmic!)Any time I come to Pike Place Market, I *must* go here. Love, love, love these piroshki's!!!! I always go down to La Panier to grab a latte then walk over to Piroshki Piroshki to grab a beef and onion ( That one is my favorite... but I like the mushroom, onion and celery as well.) then take a stroll around downtown. Feel like something sweet? Their apple cinnamon is delicious! Not toooooo sweet.. and a bit tart~ yum!This place almost always has long lines and you have to get one to go usually. They do have a counter where you can sit and eat, but it sits three-four people tops.I always bring piroshki-virgin's here... and they always end up loving it! The aromas alone make this Pike Place Market spot worth a visit. &#160;It seems that every time I am in the area there is a line-up outside of Piroshky Piroshky. &#160;This is a testament not only to their popularity, but also the size of the shop. &#160;By the time you have grabbed your pastry and paid, you have already fallen out of the other side of the shop. &#160;It is definitely small, but plenty big enough to serve its tasty purpose.With an extensive selection of meat, vegetarian and sweet choices, it is easy to find a piroshky to enjoy. &#160;Being morning and all, we went with a breakfast-y savory piroshky, Spinach Egg &amp; Cheese, and a sweet one, the Apple Cinnamon Roll. &#160;In the Spinach Egg &amp; Cheese, the pastry dough was perfectly tender and flaky, while the filling was still moist and abundant. &#160;Surprisingly filling for its size.On the sweet side of things, the Apple Cinnamon Roll appeared to be composed of a long rope of pastry dough wound in a spiral with wedges of apple inserted at frequent intervals and the whole thing lined with plenty of butter, brown sugar and cinnamon. &#160;The end result is like a cinnamon roll on a former-Soviet, Olympic steroid program with plenty of gooey cinnamon-sugar filling and tender but crisp apples baked just right.I do enjoy Piroshky Piroshky just how it is, but if they could somehow manage to serve these piping-hot out of the oven, I think that would make these pastries just about perfect. &#160;At about $3.00 to $4.00 per piroshky, this is a infrequent treat for me, but definitely worth a try for any lover of fresh-baked goods. Yummy. Yummy.This place is great, perfect for a snack while at the market.I got the cheese, onion and potato pirosky. It was soooo tasty and delicious. The outer bread reminded me of a bagel with a touch of sweetness. The inside is potatoey, cheesy goodness.My husband got the meat pirosky and he loved it.The service is super-friendly. The inside is very small with like 4 seats or so. I can't wait to go back here!! I was wary of this place given my experience with Russian cuisine while in Russia (no offense to Russians - the country was great, just the food didn't seem to agree with my stomach). However, my friend convinced me it was good. I was pleasantly surprised. The place is TINY and is pretty much standing room only unless you're lucky enough to grab 1 of the 4 bar stools available. The baked goods are fresh and tasty, and they'll heat it up for you. I had an apple cinnamon roll while my friend had a ham and cheese piroshky. Before we left, I got a marzipan roll (filled with almond paste) to go for the hubbie. Yum! Chocolate and cream cheese pinwheel, you have my heart.You also have a name, but I don't remember it...all I remember is how wonderful you tasted. And I remember how you looked, dotted with cream cheese swirls, a ribbon of smooth chocolate ganache, pastry arms gracefully akimbo.Here's a photo for our gentle readers: http://www.piroshkybaker&#8230;For our not so gentle readers: take this tasty creation home. It's like sex...addictive, seductive, wonderfully breathtaking. With a couple more calories. So i walk by here every morning on my way to work when they are just beginning to open and making their piroshkys and getting started for the day. I just about Die every single time it smells so good. I truly believe that eating too many of these things are going to be the death of me one day. The apple cinnamon ones are ridiculous out-of-this-world goood. As are the Smoked salmon ones. And all of them for that matter.For those of you who are afraid to try new foods and think they look alittle or sound funky and have never tried them? You are Crazy and should go eat one. Like right now. Great, quick eats while walking through the market; and a good variety of pastries, meat Piroshkys, and veggie Piroshkys. This place is a small shop filled with a variety of baked goodies. &#160;The outer shell of the piroshky is so flakey and yummy. &#160;I've made croissants before so I know how difficult it is to get just the right amount of flakiness. &#160;Overwork the dough and it'll easily destroy the flakiness. &#160;That was not a problem here. &#160;They must take extra care when rolling out the dough. &#160;I had the beef and onion piroshky and I enjoyed it a lot more than what my bf ordered which was the chicken, mushroom and rice. &#160;It had a peculiar seasoning that didn't agree with my tastebuds. &#160;Overall it was a wonderful treat to eat by the waterfront. &#160;Be sure to grab napkins as the flakes get all over the place. As a vegetarian it is hard to find convenient, and more importantly, good tasting food on the go. &#160;Piroshky Piroshky is FAB! &#160;Meat lovers, don't despair, I have heard nothing but great things about the meat versions, I have only had their vegetable filled piroshkies but there are so many different kinds and this is one of those places where you just can't go wrong no matter what you get. &#160;It's super fast and besides, $3.50 for a delicious, filling but not over the top lunch? &#160;Beat that, I dare you. &#160;There aren't that many foods that I crave just hearing the name, but mmmmmmm....piroshky! Delectable! &#160;I never fail to herd out of town guests past Piroshky Piroshky. &#160;It is as much a part of the Seattle Public Market landscape as the flying fish and the original Starbucks. Smoked salmon pate piroshkis...I would eat them every day, but I'm only in Seattle twice a month. You must try one (or five). They're adorable (shaped like a fish) and are so tasty. They also have a great selection of desserts--my favorite is the apple cinnamon roll, though the white chocolate cherry piroshki is great, too. Note that the place is teeny tiny, and there's only room to sit down at the four-seat counter if you're there at an off time (like a weekday morning). i'm craving one of those warm, sweet doughy creatures with the delectable savory centers. yum yum. The Salmon Pate is THE BOMB. You must, must try it. &#160;Not too smokey salmon, cream cheese, and finely chopped onion-only $4.50. The pastries here are pretty much weapons of mass destruction and should have been used in the Cold War. I'd go for the Beef and Onion for my weapon of choice here. It's small and filled with piroshkies! &#160;My friend and I split two - the spinach, egg, and cheese and also the potato, mushroom, and onion. &#160;They were served warm and were so soft, fresh, and delicious. &#160;Honestly, I could have eaten probably six more! &#160;The menu is broken into three parts: meat, vegetarian, and sweets. &#160;This place was featured on the Travel Channel's No Reservations with Anthony Bourdain when he visited the Pacific Northwest. Possibly the highlight of my trip to Seattle. If they had these things in Chicago I'd go there every week! A must have when your down at pike place. My favorite is the cheese and green onion piroshky. It's yummy! I've tried the beef and cheese when it came right out of the oven which was absolutely delicious!! They aren't as good if they've been sitting a while but usually there's enough business to keep them coming fresh out of the oven. Yum I think I want one right now... Not really much more to say other than I agree with everyone else. &#160;I bought a variety of six or so and the family loved eating them while trolling Pikes. &#160;The cheese roll is great! I'd never had a Piroshky before this, never even heard of one - but I am so glad I went to Piroshky Piroshky while I was visiting Seattle. There is nothing like meat and veggie filled pastries. This place has some tasty eats! If you're walking by its definitely worth a stop . If you go to Pike Place Market and don't enter Piroshky Piroshky, you're crazy. Aww yeah... the cheese rolls, the ones shaped like cinnamon rolls, are so good. &#160;And bonus points for the cute yellow paper sleeves they come in. &#160;I think this place is way more fun and tastier and has more delicious aromas than the piroshky place on Broadway. Yum Yum! They have all sorts of piroshky here that it's difficult picking what I want. It's not too heavy since the outside is light and crispy. I recommend eating it warm. I don't to leave a full length review here, it IS that good! Not many other foods are as easy to stuff your face while strolling the market! Man, that girl at the counter was KAYUTE!The cheese piroshky and the apple cinnamon roll was also not that bad. My sister said the cheese one reminded her of pizza, and embarrassingly enough (being a tourist and all), passed the place and gleefully shouted, \"Oh look! Innovative baked manapuas!\" like some Hawai'i hick.Well, these crafty little effers excited my little mouth, so much so that I ignored the loud Koreans eyeing my primo seat on the tiny counter. For so much business, you would think that the staff would be a little bit bitchy, but no go. I was apparently standing in front of the Russian lady proprietor and blocking her way, and all she did was laugh it off in the sweetest way possible. It may seem expected and ordinary to Americans, but coming from a half-life in Korea, that was pretty damn special and made my day completely.It made me feel, a former surly Coldstone ice cream bitch, quite ashamed.And, for the record (to idiot hicks like me), it does taste like a baked manapua with scrumptious filling."),
      "categories":"Russian,Restaurants",
      "zip_code":"98101",
      "review_count":45,
      "avg_rating":4.565217391304348}
  ]
}

parent_v1 = 'projects/%s/models/%s/versions/%s' % (PROJECT, 'hygiene', 'v1')
parent_v2 = 'projects/%s/models/%s/versions/%s' % (PROJECT, 'hygiene', 'v2')

response_v1 = api.projects().predict(body=request_data, name=parent_v1).execute()
print("Hygiene model version 1: ")
print "response={0}".format(response_v1)

response_v2 = api.projects().predict(body=request_data, name=parent_v2).execute()
print("Hygiene model version 2: ")
print "response={0}".format(response_v2)

Hygiene model version 1: 
response={u'predictions': [{u'source': u'1', u'class': 0, u'prob': [0.9659563899040222, 0.034043584018945694]}, {u'source': u'1', u'class': 0, u'prob': [0.9299690127372742, 0.07003098726272583]}, {u'source': u'0', u'class': 1, u'prob': [0.14486263692378998, 0.8551374077796936]}, {u'source': u'1', u'class': 0, u'prob': [0.6034363508224487, 0.3965635597705841]}]}
Hygiene model version 2: 
response={u'predictions': [{u'source': u'0', u'class': 1, u'prob': [0.002501737093552947, 0.9974982142448425]}, {u'source': u'1', u'class': 0, u'prob': [1.0, 1.5578968187255526e-37]}, {u'source': u'1', u'class': 0, u'prob': [1.0, 1.163709021723483e-13]}, {u'source': u'1', u'class': 0, u'prob': [0.9266756176948547, 0.07332438975572586]}]}
